# Supervised Learning for HR Separations
## November 4, 2015
<hr>

### 1. Go back to the 'raw' data so I can do the transformation in a pipe-oriented way from the start
* start with histret, histcurr, empl and bene (<font color =red> NOPE</font>)
* actually load from employees_after2001_raw.csv
* 
#### define the source repo
* '/home/kesj/lib/repo/'

### 2. Define the correct working directories
* '/data/discovery/hrsepara/core/' for HDFS
* '/data/discovery/hrsepara/staging/eda' and '/home/kesj/working/hrsepara/eda/' for HDFS and LFS on phd

### load from
* em2002 = pd.read_csv('employees_after2001_raw.csv')



In [3]:
coredir = '/data/discovery/hrsepara/core/'
stgdir1 = '/data/discovery/hrsepara/staging/eda'
stgdir1local = '/home/kesj/work/hrsepara/eda'
stgdir2local = '/home/kesj/work/hrsepara/proc'
repodir = '/home/kesj/lib/repo/'

In [4]:
### load the basic files
import os,subprocess,sys
import pandas as pd
import numpy as np
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import chain
import random
import seaborn as sns
from glob import glob
plt.style.use('ggplot')
from sklearn import cross_validation

## Plan of attack:
1. read in the data, split it, munge it as before
    * add retired variable
    * remove one case where person is 'retired' but has status of 0. 
    * save this transformed file:
2. do a simple survival analysis on the reduced set of variables
3. transform into the temporal splits and make predictions

In [5]:
os.chdir(repodir)
import bear.bear as br

# create a few additional target columns
* retired
* separated
* invol_sep


### remove the one case where a person is 'retired' but has a status of 0. 

In [6]:
## save this dataframe
os.chdir(stgdir2local)
#em2002.to_csv('after2001_keyed.csv',index=False)

### Generic processing steps
1. define set of columns to keep for analysis
    * remove historical ones & Rate
2. convert dates to timestamps and calculate the age/tenure/hire-age
3. convert indicators
4. convert BOX1

In [25]:
em2002 = pd.read_csv('after2001_keyed.csv',dtype={'EEO1CODE':np.str,'EMPL_CLASS':np.str,'SHIFT':np.str,
                                                 'VOLINVOL':np.str,'LOCATION':np.str})

In [26]:
all_cols = em2002.columns.tolist()
print "starting with {0} columns.".format(len(all_cols))

starting with 185 columns.


In [28]:
## look at historical columns and drop them
historical_cols = []
for i in xrange(2,11):
    #print str(i)
    hcols = [a for a in all_cols if a.endswith(str(i))]
    historical_cols+=hcols
print "{0} historical columns will be omited".format(len(historical_cols) )


97 historical columns will be omited


In [29]:
all_cols2 = list(set(all_cols)-set(historical_cols))
all_cols2.remove('RATE1') # drop RATE1, it is collinear with BOX1
print len(all_cols2)

87


In [30]:
mos_cols = [x for x in all_cols2 if x.endswith("MOS")]
print len(mos_cols), mos_cols
date_cols = [x for x in all_cols2 if x.endswith('DT')]
date_cols.append('BIRTHDATE')
date_cols.append('Tenure_tdelta')
date_cols.append('Age_tdelta')
print len(date_cols), date_cols

7 ['CUR_LOC_MOS', 'CUR_EFUNC_MOS', 'CUR_JOB_MOS', 'CUR_GRADE_MOS', 'TELE_MOS', 'CUR_DEPT_MOS', 'CUR_FUNC_MOS']
8 ['SERVICE_DT', 'MAR_STATUS_DT', 'HIRE_DT', 'TERMINATION_DT', 'LAST_HIRE_DT', 'BIRTHDATE', 'Tenure_tdelta', 'Age_tdelta']


# convert ages and tenures

In [31]:
def calculate_years(timestamp1,timestamp2,days_in_year =365.25):
    number_of_years = (timestamp2 - timestamp1).days/days_in_year
    return number_of_years


In [32]:
##  calculate Age_years and Tenure_years
em2002[['hire_tstmp','birth_tstmp','term_tstmp']] = em2002[['HIRE_DT','BIRTHDATE','TERMINATION_DT']].applymap(lambda x: pd.to_datetime(datetime.strptime(x,'%d%b%Y')))
em2002['Age_years']=em2002[['birth_tstmp','term_tstmp']].apply(lambda x: calculate_years(x[0],x[1]),axis=1)
em2002['Tenure_years']=em2002[['hire_tstmp','term_tstmp']].apply(lambda x: calculate_years(x[0],x[1]),axis=1)


In [33]:
hire_age_tdelta = em2002['hire_tstmp']-em2002['birth_tstmp']
# convert to days, months, or years
em2002['hire_age'] = hire_age_tdelta/np.timedelta64(1,'Y')

In [34]:
tstmp_cols = [x for x in em2002.columns if x.endswith('tstmp')]
print len(tstmp_cols), tstmp_cols

useful_time_cols = []
useful_time_cols.append('Tenure_years')
useful_time_cols.append('Age_years')
useful_time_cols.append('hire_age')
print len(useful_time_cols), useful_time_cols

3 ['hire_tstmp', 'birth_tstmp', 'term_tstmp']
3 ['Tenure_years', 'Age_years', 'hire_age']


In [35]:
all_cols3 = list(set(all_cols2)-set(mos_cols)-set(date_cols))
all_cols3 = list(set(all_cols3).union(set(tstmp_cols)))
all_cols3+=useful_time_cols# redoing keeping tstmp columns
print "now there are {0} total columns to consider.".format(len(all_cols3))

now there are 80 total columns to consider.


### Look for missing values in these columns to correct

In [36]:
br.get_columns_with_all_nulls(em2002[all_cols3]) # good no columns are fully empty

[]

In [41]:
em2002[['MERIT1','SAL1','PERF1','SAL2']].tail()

,MERIT1,SAL1,PERF1,SAL2
134261,3960.25,77298.13,2640.16,73337.88
134262,6106.72,84398.05,4071.15,73169.47
134263,3752.34,73240.21,2501.56,69487.87
134264,3120.00,68120.00,2080.00,0.00
134265,1748.25,57248.25,1165.50,0.00


In [42]:
(em2002['SAL1']-em2002['SAL2']).tail()

134261     3960.25
134262    11228.58
134263     3752.34
134264    68120.00
134265    57248.25
dtype: float64

### deal with missing values

In [43]:
missing_value_cols  = br.get_columns_with_nulls(em2002[all_cols3])
print len(missing_value_cols)

22


In [ ]:
[(a, em2002[a].dtype) for a in missing_value_cols]

## fill in missing values

In [ ]:
## strings
em2002.fillna({'ACTRES1':'UNKNOWN','LOC_STATE':'XX','STATE':'XX','LOC_CITY':'UNKNOWN','SKEY':'UNKNOWN',
               'GRADE':'XX','JOB_FAMILY':'XX','POSTAL_SFI':'XXX','ADDRESS1':'UNKNOWN','VOLINVOL':'N/A',
              'JOB_FUNCTION':'XX'},inplace=True)
em2002.fillna({'BOX1':0,'HAVE_INS':'N','HAVE_DEP':'N'},inplace=True)
## floats
em2002.fillna({'DEPENDENT_CNT':0,'TOTAL_RPT_CNT':0,'EXTFUNC_CNT':0.0,'MERIT1':0.0,'PERF1':0.0,'SAL1':0.0,
              'FUNC_CNT':0.0,'DIRECT_RPT_CNT':0.0,'ADDRCNT1':0.0},inplace=True)

br.get_columns_with_nulls(em2002[all_cols3])

## identify the response/indicator variables

In [ ]:
indicator_dict = {'INTERN': {'N':0,'Y':1},
                  'REMOTE': {'N':0,'Y':1}, 'FULLPART1': {'N':0,'Y':1},
                 'RELOCATE_ALL_SFI' : {'N':0,'Y':1},
                'HUBIND':{'N':0,'Y':1},'SUPV_DIFF_LOC':{'N':0,'Y':1},
     'HAVE_DEP': {'N':0,'Y':1},'HAVE_INS':{'N':0,'Y':1},'PARTFULL1':{'N':0,'Y':1},
    'REMOTE_SUPV':{'N':0,'Y':1}, 'COMP_FREQUENCY': {'A':1,'H':0}, 
                  'SEX': {'M':1,'F':0},'BOX1':{'H':3,'S':2,'L':1},'SHIFT':{'N':0,'1':1,'2':2,'3':3}}
for key,value in indicator_dict.iteritems():
    em2002[key].replace(value,inplace=True)

In [ ]:
em2002['SHIFT'].replace({'1':1,'2':2,"3":3},inplace=True)

### get rid of some other columns

In [ ]:
#all_cols =em2002.columns.tolist()
#columns_to_omit = [a for a in em2002.columns if a.startswith('ACTRES')]
columns_to_omit =[]
columns_to_omit.append('ACTRES1')
columns_to_omit.append('ADDRESS1')
#columns_to_omit.append('MAR_STATUS_DT')
#columns_to_omit.append('KEY')
columns_to_omit.append('SKEY')
columns_to_omit.append('STATE') # these values are noisier than LOC_STATE
columns_to_omit.append('LOC_CITY')
#columns_to_omit.append('zip5')
columns_to_omit.append('PER_ORG')
columns_to_omit.append('POSTAL_SFI')
columns_to_omit.append('VOLINVOL')
columns_to_omit.append('MAR_STA_SNAME_SFI')
#columns_to_omit.remove('LOCATION')
columns_to_omit.append('LEGACY_DEPT_SFI')
columns_to_omit.append('TOT_MO_SERVICE_SFI')
print len(columns_to_omit)
all_cols4 = list(set(all_cols3)-set(columns_to_omit))
print len(all_cols4)

In [ ]:
col3_categorical = br.get_categorical(em2002[all_cols4])
col3_numeric = br.get_numeric(em2002[all_cols4])

In [ ]:
len(col3_categorical), len(col3_numeric), len([a for a in all_cols3 if a.endswith('tstmp')])

## save this set to a file

In [ ]:
save_file = False
if save_file:
    em2002[all_cols4].to_csv('after2001_v3.csv',index=False)


## START LOADING HERE 12.09.2015

In [7]:

em2002 = pd.read_csv('after2001_v3.csv',dtype={'EEO1CODE':np.str,'EMPL_CLASS':np.str})

In [8]:
em2002.shape

(134266, 69)

### get categorical columns


In [9]:
col3_categorical = br.get_categorical(em2002)
col3_numeric = br.get_numeric(em2002)
print len(col3_numeric), len(col3_categorical)

55 14


## Deal with categorical columns

In [10]:
#indicator_cols = []
#cat_cols2 = []
for c in col3_categorical: #cat_cols:
    nobs = len(em2002[c].unique())
    #if nobs == 2:
        #indicator_cols.append(c)
    #else:
    print c, nobs
    #    cat_cols2.append(c)
print "============"
numeric_to_categorical = ['COMPANY','DIVISION_CODE_SFI','ETHNIC_GROUP']
for c in numeric_to_categorical:
    print c, len(em2002[c].unique())

EEO1CODE 10
birth_tstmp 20171
FULL_PART_TIME 4
EMPL_CLASS 6
FLSA_STATUS 3
LOC_STATE 52
GRADE 125
JOB_FAMILY 1448
hire_tstmp 8881
LOCATION 13030
term_tstmp 4353
JOB_FUNCTION 19
EMPL_TYPE 5
LOC_TYPE_DESCR_SFI 31
COMPANY 6
DIVISION_CODE_SFI 9
ETHNIC_GROUP 8


In [57]:
len(em2002['FUNC_ID_SFI'].unique())

53

In [66]:

cat_cols_to_dummy_encode = ['EMPL_TYPE','EMPL_CLASS','EEO1CODE','FULL_PART_TIME','FLSA_STATUS','LOC_TYPE_DESCR_SFI',
                           'COMPANY','DIVISION_CODE_SFI','ETHNIC_GROUP','JOB_FUNCTION']
my_label_encode_cols = ['JOB_FAMILY', 'GRADE', 'FLOR_SFI', 'FUNC_ID_SFI', 'EXT_FUNC_ID_SFI',
                        'JOBCODE', 'LOC_STATE']

In [67]:
[(col,em2002[col].dtype) for col in cat_cols_to_dummy_encode]

[('EMPL_TYPE', dtype('O')),
 ('EMPL_CLASS', dtype('O')),
 ('EEO1CODE', dtype('O')),
 ('FULL_PART_TIME', dtype('O')),
 ('FLSA_STATUS', dtype('O')),
 ('LOC_TYPE_DESCR_SFI', dtype('O')),
 ('COMPANY', dtype('int64')),
 ('DIVISION_CODE_SFI', dtype('int64')),
 ('ETHNIC_GROUP', dtype('float64')),
 ('JOB_FUNCTION', dtype('O'))]

## identify meaningless columns for modeling
* potential responses: retired, status
* timestamps (3)
* KEY, SKEY, ACTRES1, MAR_STA_SNAME_SFI, LOC_TYPE_DESCR_SFI

## Take a look at the numeric columns

In [68]:
true_numeric = list(set(col3_numeric)-set(numeric_to_categorical)-set(my_label_encode_cols))
true_numeric.sort()
len(true_numeric)

48

In [13]:
[(col,em2002[col].nunique()) for col in true_numeric]

[('ADDRCNT1', 5),
 ('ANNUAL_RT', 95474),
 ('Age_years', 18247),
 ('BOX1', 4),
 ('COMP_FREQUENCY', 2),
 ('DEPENDENT_CNT', 28),
 ('DEPTCNT1', 8),
 ('DIRECT_RPT_CNT', 173),
 ('EFUNCCNT1', 5),
 ('EXTFUNC_CNT', 6518),
 ('FLOORCNT1', 6),
 ('FTE', 77),
 ('FTPTCNT1', 3),
 ('FULLPART1', 2),
 ('FUNCCNT1', 4),
 ('FUNC_CNT', 7807),
 ('GRADECNT1', 5),
 ('HAVE_DEP', 2),
 ('HAVE_INS', 2),
 ('HUBIND', 2),
 ('INTERN', 2),
 ('JOBCNT1', 7),
 ('KEY', 134266),
 ('LOCCNT1', 7),
 ('LOCSTCNT1', 4),
 ('MAX_RT_ANNUAL', 1002),
 ('MERIT1', 74185),
 ('MIN_RT_ANNUAL', 1255),
 ('PARTFULL1', 2),
 ('PERF1', 78546),
 ('PTFTCNT1', 3),
 ('REH_CNT', 18),
 ('RELOCATE_ALL_SFI', 2),
 ('RELO_STATE_CNT_SFI', 55),
 ('REMOTE', 2),
 ('REMOTE_SUPV', 2),
 ('SAL1', 95474),
 ('SEX', 2),
 ('SHIFT', 4),
 ('STD_HOURS', 115),
 ('SUPVCNT1', 12),
 ('SUPV_DIFF_LOC', 2),
 ('TOTAL_RPT_CNT', 854),
 ('Tenure_years', 14693),
 ('former', 2),
 ('hire_age', 15240),
 ('retired', 2),
 ('status', 2)]

In [15]:
br.perfect_collinearity_test(em2002[true_numeric])

ADDRCNT1           VIF = 1.0   R^2 = 0.0403    

ANNUAL_RT: PERFECT COLLINEARITY ********
['ADDRCNT1', 'FTE', 'SAL1', 'STD_HOURS']

Age_years          VIF = 906310775.4R^2 = 1.0       
BOX1               VIF = 2.4   R^2 = 0.5824    
COMP_FREQUENCY     VIF = 2.4   R^2 = 0.5789    
DEPENDENT_CNT      VIF = 2.9   R^2 = 0.6544    
DEPTCNT1           VIF = 7.3   R^2 = 0.8621    
DIRECT_RPT_CNT     VIF = 1.7   R^2 = 0.4221    
EFUNCCNT1          VIF = 7.2   R^2 = 0.8611    
EXTFUNC_CNT        VIF = 19.8  R^2 = 0.9495    
FLOORCNT1          VIF = 20.6  R^2 = 0.9514    
FTE                VIF = 632.1 R^2 = 0.9984    
FTPTCNT1           VIF = 17397.2R^2 = 0.9999    
FULLPART1          VIF = 17397.2R^2 = 0.9999    
FUNCCNT1           VIF = 6.2   R^2 = 0.838     
FUNC_CNT           VIF = 6.6   R^2 = 0.8483    
GRADECNT1          VIF = 22.5  R^2 = 0.9555    
HAVE_DEP           VIF = 4.0   R^2 = 0.7492    
HAVE_INS           VIF = 2.9   R^2 = 0.6577    
HUBIND             VIF = 3.3   R^2 = 0.6938  

ADDRCNT1              0.040298
ANNUAL_RT             1.000000
Age_years             1.000000
BOX1                  0.582429
COMP_FREQUENCY        0.578885
DEPENDENT_CNT         0.654439
DEPTCNT1              0.862119
DIRECT_RPT_CNT        0.422054
EFUNCCNT1             0.861106
EXTFUNC_CNT           0.949467
FLOORCNT1             0.951448
FTE                   0.998418
FTPTCNT1              0.999943
FULLPART1             0.999943
FUNCCNT1              0.838028
FUNC_CNT              0.848312
GRADECNT1             0.955484
HAVE_DEP              0.749163
HAVE_INS              0.657694
HUBIND                0.693786
INTERN                0.913711
JOBCNT1               0.954493
KEY                   0.197428
LOCCNT1               0.835615
LOCSTCNT1             0.955677
MAX_RT_ANNUAL         0.957170
MERIT1                0.695590
MIN_RT_ANNUAL         1.000000
PARTFULL1             0.103042
PERF1                 1.000000
PTFTCNT1              0.104258
REH_CNT               0.584472
RELOCATE

In [21]:
iterative_modeling_cols = [col for col in true_numeric]
iterative_modeling_cols.remove('status')
iterative_modeling_cols.remove('retired')
iterative_modeling_cols.remove('KEY')
iterative_modeling_cols.remove('former')
iterative_modeling_cols.remove('Tenure_years')
iterative_modeling_cols.remove('ANNUAL_RT')
iterative_modeling_cols.remove('Age_years')
len(iterative_modeling_cols)

41

 Oct. 23, 2015
## Looking at doing survival analysis for Retirement
plan of attack

1. id the minimum age of retirement, $A_R$, in our dataset and adjust it by $y_H$, the maximum horizon age..
2. use the train-test-split from above (build & evaluation sets)
3. pull out subsets from each where the following conditions are met:
    a. retired == 1  OR
    b. Age $\ge A_R - y_H$
4. calculate retire age
    * I might have to calculate this in relative terms
5. take subset of columns (minimal_cols_to_keep)
6. apply coxph to get prob of retire and weights of factors/features.
7. also try other models like Aaalen and Nelson
7. test on evaluation set for ability to predict 
    * ...
9. check for retirement rates in each of the coming years

## for retirement split based on age (50 years)

In [16]:
over50 = em2002[em2002.Age_years >= 50.0].copy()
print len(over50)
#over50.to_csv('over50.csv',index=False)

43536


In [19]:
# repeat the split

# break into evaluation and build sets
print "Starting with subset of {0} employees.".format(len(over50))
eval_fraction = 0.20
o50build, o50eval = cross_validation.train_test_split(over50,test_size=eval_fraction,random_state = 88843)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(o50eval),len(o50build))

Starting with subset of 43536 employees.
Evaluation set has 8708 employees; training set has 34828 employees.


In [20]:
from lifelines import CoxPHFitter
cf = CoxPHFitter()
#from lifelines import AalenAdditiveFitter

In [24]:
#define a function to fit and score a model with variables
def optimize_model(df,column_list,tgt_col = 'Age_years',ind_col='retired'):
    initial_concord
    my_col = [tgt_col,ind_col]
    for col in column_list:
        cf = CoxPHFitter()
        my_col.append(col)
        cf.fit(df[my_col],tgt_col,event_col=ind_col)
        print col
        cf.print_summary()
    return

In [70]:
o50build.shape

(34828, 69)

In [59]:
%time
tmp_col = ['SEX','hire_age','SAL1','MERIT1','HAVE_INS','HAVE_DEP','PERF1','BOX1']
optimize_model(o50build,tmp_col)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 20 µs
SEX
n=34828, number of events=12036

          coef  exp(coef)  se(coef)          z         p  lower 0.95  upper 0.95     
SEX -5.138e-02  9.499e-01 9.173e-03 -5.602e+00 2.123e-08  -6.936e-02  -3.340e-02  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Concordance = 0.529
hire_age
n=34828, number of events=12036

               coef  exp(coef)  se(coef)          z         p  lower 0.95  upper 0.95     
SEX      -7.477e-02  9.280e-01 9.181e-03 -8.144e+00 3.826e-16  -9.277e-02  -5.677e-02  ***
hire_age -3.801e-01  6.838e-01 8.596e-03 -4.422e+01 0.000e+00  -3.970e-01  -3.633e-01  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Concordance = 0.622
SAL1
n=34828, number of events=12036

               coef  exp(coef)  se(coef)          z         p  lower 0.95  upper 0.95     
SEX      -4.896e-02  9.522e-01 1.042e-02 -4.698e+00 2.625e-06  -6.938e-02  -2.853e-02  ***
hire_age -3.943e

In [66]:
o50build[['retired','Age_years','SEX','hire_age']].head()

,retired,Age_years,SEX,hire_age
113856,0,56.150582,1,39.718817
78215,0,54.910335,0,17.615694
77668,0,57.634497,0,21.059981
78516,0,55.523614,1,25.676092
18773,1,57.593429,0,35.529819


### build a coxproporitional hazard model based upon:
* Sex, hire_age, SAL1, MERIT1, HAVE_INS, & HAVE_DEP (model_A)

In [71]:
cf_modA = CoxPHFitter()
cf_modA.fit(o50build[['retired','Age_years','SEX','hire_age','SAL1','MERIT1','HAVE_INS','HAVE_DEP']],'Age_years',event_col='retired')

<lifelines.CoxPHFitter: fitted with 34828 observations, 22792 censored>

In [72]:

cf_modA.print_summary()

n=34828, number of events=12036

               coef  exp(coef)  se(coef)          z         p  lower 0.95  upper 0.95     
SEX      -7.390e-02  9.288e-01 1.072e-02 -6.894e+00 5.412e-12  -9.492e-02  -5.289e-02  ***
hire_age -4.020e-01  6.690e-01 9.029e-03 -4.453e+01 0.000e+00  -4.197e-01  -3.843e-01  ***
SAL1     -4.106e-02  9.598e-01 1.032e-02 -3.978e+00 6.940e-05  -6.129e-02  -2.083e-02  ***
MERIT1   -7.452e-02  9.282e-01 8.304e-03 -8.973e+00 2.874e-19  -9.079e-02  -5.824e-02  ***
HAVE_INS -1.390e-01  8.703e-01 1.130e-02 -1.230e+01 9.614e-35  -1.611e-01  -1.168e-01  ***
HAVE_DEP  7.221e-02  1.075e+00 1.107e-02  6.521e+00 6.993e-11   5.050e-02   9.392e-02  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Concordance = 0.640


In [73]:
cf_modA.summary

,coef,exp(coef),se(coef),z,p,lower 0.95,upper 0.95
SEX,-0.073902,0.928763,0.010719,-6.894337,5.411648e-12,-0.094916,-0.052888
hire_age,-0.401999,0.668981,0.009029,-44.525426,0.000000e+00,-0.419699,-0.384300
SAL1,-0.041061,0.959771,0.010321,-3.978329,6.940134e-05,-0.061294,-0.020827
MERIT1,-0.074516,0.928193,0.008304,-8.973438,2.874016e-19,-0.090795,-0.058237
HAVE_INS,-0.138952,0.870269,0.011301,-12.295185,9.613617e-35,-0.161108,-0.116797
HAVE_DEP,0.072214,1.074885,0.011074,6.520809,6.992936e-11,0.050504,0.093924


In [75]:
cf_modA.predict_expectation(o50build[['retired','Age_years','SEX','hire_age','SAL1','MERIT1','HAVE_INS','HAVE_DEP']])

,0
113856,63.670215
78215,59.633871
77668,60.440856
78516,61.882317
18773,61.464658
35635,63.550532
106006,61.962650
120660,60.821555
26127,62.186395
29727,60.010272


In [185]:
cf_modA2 = CoxPHFitter()
cf_modA2.fit(o50build[['retired','Age_years','SEX','Tenure_years','SAL1','MERIT1','HAVE_INS','HAVE_DEP']],'Age_years',event_col='retired')

<lifelines.CoxPHFitter: fitted with 34828 observations, 22792 censored>

In [187]:
cf_modA2.summary

,coef,exp(coef),se(coef),z,p,lower 0.95,upper 0.95
SEX,-0.108925,0.896798,0.010583,-10.292705,7.601017e-25,-0.129671,-0.088179
Tenure_years,0.137737,1.147673,0.008770,15.705921,1.377616e-55,0.120545,0.154929
SAL1,0.047574,1.048724,0.009345,5.091006,3.561682e-07,0.029254,0.065893
MERIT1,-0.076740,0.926131,0.007591,-10.109597,5.008995e-24,-0.091621,-0.061859
HAVE_INS,-0.096487,0.908022,0.011276,-8.556581,1.162642e-17,-0.118593,-0.074381
HAVE_DEP,0.084767,1.088463,0.011090,7.643436,2.115004e-14,0.063026,0.106508


In [ ]:
concordance_index(self.durations,
                                        -self.predict_partial_hazard(self.data).values.ravel(),
                                        self.event_observed))

In [40]:
cf.predict_partial_hazard(o50build).values.ravel()

array([ 0.70217538,  2.24861417,  1.52582098, ...,  1.62656307,
        0.7263484 ,  1.26909467])

In [53]:
from lifelines.utils import concordance_index 
concordance_index(cf.durations.values,cf.predict_expectation(o50build[['retired','Age_years','SEX','HAVE_INS','hire_age']]).values.ravel(),cf.event_observed.astype(int).values )

0.50206968819783226

In [49]:
cf.event_observed.astype(int).values

array([0, 0, 0, ..., 1, 1, 1])

In [58]:
expectations = cf.predict_expectation(o50build[['retired','Age_years','SEX','HAVE_INS','hire_age']])
expectations.head()

,0
113856,63.689903
78215,59.691828
77668,60.745581
78516,61.719523
18773,61.391227


In [57]:
cf.durations[cf.event_observed]

9823     50.206708
30383    50.354552
42472    50.535250
53833    50.866530
15494    51.104723
27439    51.362081
27661    51.592060
18385    51.624914
24426    51.931554
43488    51.969884
46548    52.520192
49802    52.681725
14589    52.884326
52843    53.199179
62250    53.273101
26108    53.423682
20521    53.464750
30990    53.514031
6063     53.670089
29719    53.809719
40242    54.113621
53900    54.924025
33865    54.932238
23399    54.978782
53832    54.997947
53233    55.000684
51036    55.000684
13944    55.000684
10032    55.000684
49188    55.000684
           ...    
6801     77.527721
45655    77.648186
37332    77.730322
51117    77.982204
18962    78.384668
20456    78.767967
6992     78.888433
6681     78.891170
42413    78.893908
6        79.080082
14       79.189596
37333    79.252567
18       79.685147
16       79.761807
24592    80.416153
9        80.804928
41090    80.887064
33835    80.999316
36300    81.481177
9423     81.705681
46384    81.812457
4805     82.

In [ ]:
#surv_minimal_cols_to_keep = ['Age_years','Tenure_years','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP']

In [176]:
retire_surv_cols1 = ['retired','Age_years','hire_age','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP']
cf2 = CoxPHFitter()
cf2.fit(o50build[retire_surv_cols1], 'Age_years', event_col='retired')

cf2.print_summary()

n=34828, number of events=12036

               coef  exp(coef)  se(coef)          z         p  lower 0.95  upper 0.95     
hire_age -4.004e-01  6.700e-01 9.157e-03 -4.373e+01 0.000e+00  -4.184e-01  -3.825e-01  ***
SAL1     -4.233e-02  9.586e-01 1.045e-02 -4.051e+00 5.109e-05  -6.281e-02  -2.184e-02  ***
MERIT1   -7.657e-02  9.263e-01 8.356e-03 -9.164e+00 5.018e-20  -9.295e-02  -6.019e-02  ***
PERF1     7.019e-03  1.007e+00 1.117e-02  6.284e-01 5.297e-01  -1.488e-02   2.892e-02     
BOX1      9.885e-03  1.010e+00 1.116e-02  8.855e-01 3.759e-01  -1.200e-02   3.177e-02     
SEX      -7.263e-02  9.299e-01 1.086e-02 -6.687e+00 2.277e-11  -9.392e-02  -5.134e-02  ***
HAVE_INS -1.404e-01  8.690e-01 1.135e-02 -1.237e+01 4.010e-35  -1.627e-01  -1.181e-01  ***
HAVE_DEP  7.144e-02  1.074e+00 1.109e-02  6.443e+00 1.175e-10   4.970e-02   9.317e-02  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Concordance = 0.640


In [70]:
cf.hazards_

,SEX,hire_age
coef,-0.074769,-0.38012


In [177]:
cf2.hazards_

,hire_age,SAL1,MERIT1,PERF1,BOX1,SEX,HAVE_INS,HAVE_DEP
coef,-0.400404,-0.042327,-0.076573,0.007019,0.009885,-0.07263,-0.140408,0.071437


## pickle my result

In [88]:
import pickle
pickle.dump( cf_modA, open( "retirement_sfA.pkl", "wb" ) )

In [91]:
## now load it
ret_model = pickle.load(open('retirement_sfA.pkl','rb'))

In [92]:
ret_model.hazards_

,SEX,hire_age,SAL1,MERIT1,HAVE_INS,HAVE_DEP
coef,-0.073902,-0.401999,-0.041061,-0.074516,-0.138952,0.072214


## predict results for o50eval

In [84]:
o50eval[['Age_years','retired']].head()

,Age_years,retired
9337,55.077344,1
68349,56.380561,0
36915,52.068446,0
79474,51.501711,0
79226,50.439425,0


In [86]:
cf_modA.predict_survival_function(o50eval.head()).ix[58.07:]

,9337,68349,36915,79474,79226
event_at,,,,,
58.072553,7.585346e-01,0.818153,7.636698e-01,0.815065,0.814911
58.075291,7.582935e-01,0.817965,7.634331e-01,0.814873,0.814719
58.078029,7.580524e-01,0.817776,7.631962e-01,0.814681,0.814527
58.080767,7.578916e-01,0.817650,7.630383e-01,0.814553,0.814399
58.083504,7.578113e-01,0.817587,7.629594e-01,0.814490,0.814335
58.086242,7.576505e-01,0.817461,7.628015e-01,0.814362,0.814207
58.088980,7.575701e-01,0.817398,7.627225e-01,0.814298,0.814143
58.091718,7.574092e-01,0.817272,7.625645e-01,0.814170,0.814015
58.094456,7.570876e-01,0.817020,7.622485e-01,0.813914,0.813759


In [72]:
cf2_pred = cf2.predict_survival_function(o50eval[retire_surv_cols1[2:]].values)
cf2_pred.index.name='years'
cf2_pred.reset_index(inplace=True)
cf2_pred.head()

,years,0,1,2,3,4,5,6,7,8,...,8698,8699,8700,8701,8702,8703,8704,8705,8706,8707
0,0.000000,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,50.001369,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,50.004107,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,50.006845,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,50.009582,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


## dec 17, 2015
replace hire_age with Tenure_years

In [188]:
retire_surv_cols1 = ['retired','Age_years','Tenure_years','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP']
cf3 = CoxPHFitter()
cf3.fit(o50build[retire_surv_cols1], 'Age_years', event_col='retired')

cf3.print_summary()

n=34828, number of events=12036

                   coef  exp(coef)  se(coef)          z         p  lower 0.95  upper 0.95     
Tenure_years  1.301e-01  1.139e+00 8.871e-03  1.467e+01 1.016e-48   1.127e-01   1.475e-01  ***
SAL1          4.112e-02  1.042e+00 9.696e-03  4.241e+00 2.228e-05   2.211e-02   6.012e-02  ***
MERIT1       -8.619e-02  9.174e-01 7.356e-03 -1.172e+01 1.043e-31  -1.006e-01  -7.177e-02  ***
PERF1         4.712e-02  1.048e+00 8.894e-03  5.298e+00 1.170e-07   2.969e-02   6.456e-02  ***
BOX1          2.355e-02  1.024e+00 1.065e-02  2.211e+00 2.701e-02   2.673e-03   4.442e-02    *
SEX          -1.072e-01  8.984e-01 1.070e-02 -1.002e+01 1.252e-23  -1.282e-01  -8.622e-02  ***
HAVE_INS     -1.021e-01  9.029e-01 1.133e-02 -9.012e+00 2.020e-19  -1.243e-01  -7.990e-02  ***
HAVE_DEP      8.121e-02  1.085e+00 1.111e-02  7.312e+00 2.626e-13   5.944e-02   1.030e-01  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Concordance = 0.569


In [189]:
cf2.print_summary()

n=34828, number of events=12036

               coef  exp(coef)  se(coef)          z         p  lower 0.95  upper 0.95     
hire_age -4.004e-01  6.700e-01 9.157e-03 -4.373e+01 0.000e+00  -4.184e-01  -3.825e-01  ***
SAL1     -4.233e-02  9.586e-01 1.045e-02 -4.051e+00 5.109e-05  -6.281e-02  -2.184e-02  ***
MERIT1   -7.657e-02  9.263e-01 8.356e-03 -9.164e+00 5.018e-20  -9.295e-02  -6.019e-02  ***
PERF1     7.019e-03  1.007e+00 1.117e-02  6.284e-01 5.297e-01  -1.488e-02   2.892e-02     
BOX1      9.885e-03  1.010e+00 1.116e-02  8.855e-01 3.759e-01  -1.200e-02   3.177e-02     
SEX      -7.263e-02  9.299e-01 1.086e-02 -6.687e+00 2.277e-11  -9.392e-02  -5.134e-02  ***
HAVE_INS -1.404e-01  8.690e-01 1.135e-02 -1.237e+01 4.010e-35  -1.627e-01  -1.181e-01  ***
HAVE_DEP  7.144e-02  1.074e+00 1.109e-02  6.443e+00 1.175e-10   4.970e-02   9.317e-02  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Concordance = 0.640


In [ ]:
import scipy.interpolate
def survival_predictions(edf,psf = cf_pred):
    for employee_id,age in enumerate(ages):
        prior_idx = np.where(psf.years<age+1.0)
        posterior_idx = np.where(psf.years>age+1.0)
        x= [psf.ix[prior_idx]['years'], psf.ix[posterior_idx]['years']]
        y= [psf.ix[prior_idx][employee_id],psf.ix[posterior_idx][employee_id]]
        print x,y, employee_id, age
    ## now interpolate these
        y_interp = scipy.interpolateinterp1d(x,y)
        print y_interp
    return y_interp
    

In [ ]:
def get_survival_prediction(edf,psf=cf_pred, age_col='Age_years',yr_vals=[1,2,3,4,5]):
    
    pred_col_names = ['proba_'+str(y) for y in yr_vals]
    df = pd.DataFrame(index=edf.index,columns=pred_col_names,dtype=float)
    print pred_col_names
    for employee_id,age in enumerate(edf['Age_years']):
        if employee_id % 1000 ==0 :
            print employee_id, age
            
        prob_vals =np.ones((len(yr_vals),))
        #print prob_vals,"\n"
        
        for idx,year in enumerate(yr_vals):
            ck_yr = age+year
            #if ck_yr in psf.years:
            #    proba = psf[psf.years==ck_yr][employee_id]
            #else:
            #proba=np.nan
            
            try:
                prior_idx = np.where(psf.years < ck_yr)[0][-1]
                posterior_idx = np.where(psf.years > ck_yr)[0][0]
                x= [psf.ix[prior_idx]['years'], psf.ix[posterior_idx]['years']]
                y= [psf.ix[prior_idx][employee_id],psf.ix[posterior_idx][employee_id]]
                #print x,y, year
                ## now interpolate these
                y_interp = scipy.interpolate.interp1d(x,y)
                proba = y_interp(ck_yr)
            except IndexError:
                proba = psf.ix[np.where(psf.years == ck_yr)[0]][employee_id]
            #print proba
            
            try:
                prob_vals[idx] = proba
                #print employee_id,age,ck_yr,proba, prob_vals[idx]
            except ValueError: # case when age not found because too young
                #lse:
                pass #print idx, prob_vals[idx]
                # just leave it as the default value of 1
                    
            #prob_vals[id]=1-proba    
            #if len(yr_vals)>1:
            #    empl_list.append(1-proba)# get prob of retirement
            #else:
            #    empl_list = 1-proba
        #try:
        df.loc[edf.index[employee_id]]=prob_vals
        #except ValueError: # case when not found (because too young)
        #    df.loc[edf.index[employee_id]]=0.0 # zero probability
            #print employee_id, edf.index[employee_id]
    df.fillna(0.0,inplace=True) # fillin missing values with zero
    df[df<0] = 0.0 #replace negative probabilities with zero
    #df.replace()
    return df


In [ ]:
cfRetProb = get_survival_prediction(o50eval,psf=cf_pred)

In [ ]:
#pred_o50eval=pd.concat([o50eval[retire_surv_cols1[:2]],retireProb_df],axis=1)
pred_o50eval_cf = pd.concat([o50eval[retire_surv_cols1[:2]],cfRetProb],axis=1)

In [ ]:
pred_o50eval_cf.head()

### assess the prob_retired at current age for eval set

In [ ]:
curr_retire_age = get_survival_prediction(o50eval,yr_vals=[0])
o50eval_curr = pd.concat([o50eval[retire_surv_cols1[:2]],curr_retire_age],axis=1)

In [ ]:
o50eval_curr.head() #proba is the probability of 'survival' so I want 1-proba for probability to retire

In [ ]:
def get_accuracy(df,true_col = 'retired', p_col = 'proba_0',thresh=0.5,verbose=False):
    ckdf = (df[p_col]<=thresh).astype('int')
    true_pos = df[ckdf==1][true_col].sum()
    false_pos= len(df[ckdf==1])-true_pos
    
    false_neg=df[ckdf==0][true_col].sum()
    true_neg =len(df[ckdf==0]) - false_neg
    if verbose:
        print "threshold = {0}".format(thresh)
        print "TP\t FP\t FN\t TN"
        print true_pos, '\t',false_pos,'\t' ,false_neg,'\t', true_neg
    
    acc = (true_pos + true_neg)/float(len(df))
    
    mcc = (true_pos*true_neg - false_pos*false_neg)/np.sqrt((true_pos+false_pos)*(true_pos+false_neg)*(true_neg+false_pos)*(true_neg+false_neg))
    if verbose:
        print "Accuracy is {0:.3f}".format(acc)
        print "Matthews correlation coefficient is {0:.3f}".format(mcc)
    tpr = true_pos/float(true_pos+false_neg)
    fpr = false_pos/float(false_pos + true_neg)
    return tpr,fpr,acc,mcc

In [ ]:
from sklearn import metrics
metrics.roc_auc_score(o50eval_curr.retired.values,1-o50eval_curr.proba_0.values)

In [ ]:
roc_vals = []
tvals = np.linspace(0,1,21)#,0.05)#:#xrange(0,1,0.05):
for t in tvals:
    roc_vals.append(get_accuracy(o50eval_curr,thresh=t))

    
roc_vals = np.array(roc_vals)

In [ ]:
roc_vals = np.array(roc_vals)
plt.plot(roc_vals[:,1],roc_vals[:,0])
plt.plot(tvals,tvals,color='k',ls='--')
plt.xlabel('FPR')
plt.ylabel('TPR')

# Repeat for Separation
* use all employees -- split into test & training

In [ ]:
# repeat the split

# break into evaluation and build sets
print "Starting with subset of {0} employees.".format(len(over50))
eval_fraction = 0.20
e2build, e2eval = cross_validation.train_test_split(em2002,test_size=eval_fraction,random_state = 88843)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(e2eval),len(e2build))

In [ ]:
sep_surv_cols1=['former','Tenure_years','Age_years','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP']
#'SEX','FTE','HAVE_INS','HAVE_DEP','SAL1']

In [ ]:
sepCF = CoxPHFitter()
sepCF.fit(e2build[sep_surv_cols1], 'Tenure_years', event_col='former')

sepCF.print_summary()



In [ ]:
sepCF.hazards_

In [ ]:
sepcf_pred = sepCF.predict_survival_function(e2eval[sep_surv_cols1[2:]].values)
sepcf_pred.index.name='years'
sepcf_pred.reset_index(inplace=True)
sepcf_pred.head()

In [ ]:
plt.plot(sepcf_pred.years, sepcf_pred[0])

In [ ]:
cfSepProb = get_survival_prediction(e2eval,psf=sepcf_pred)

In [ ]:
cfSepProb.head()

## assess the current Sep probability

In [ ]:
curryear_sep_pred = get_survival_prediction(e2eval,yr_vals=[0],psf=sepcf_pred)


In [ ]:
e2eval_sepNow = pd.concat([e2eval[sep_surv_cols1[:2]],curryear_sep_pred],axis=1)

In [ ]:
e2eval_sepNow.head()

In [ ]:
metrics.roc_auc_score(e2eval_sepNow.former.values,1-e2eval_sepNow.proba_0.values)

In [ ]:
sep_roc_vals = []
tvals = np.linspace(0,1,41)#,0.05)#:#xrange(0,1,0.05):
for t in tvals:
    sep_roc_vals.append(get_accuracy(e2eval_sepNow,thresh=t,true_col='former'))

    
sep_roc_vals = np.array(sep_roc_vals)

In [ ]:
plt.plot(sep_roc_vals[:,1],sep_roc_vals[:,0])
plt.plot(tvals,tvals,color='k',ls='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Separation ROC curve; current values e2eval')

## Examine the retirement Rates for current employees

In [ ]:
current = em2002[em2002.status==0].copy() 
np.shape(current)


### apply retirement and separation for next 5 years to this set

In [ ]:
current_retcf_pred = cf.predict_survival_function(current[retire_surv_cols1[2:]].values)
current_retcf_pred.index.name='years'
current_retcf_pred.reset_index(inplace=True)
current_retcf_pred.head()
currentcfRetProb = get_survival_prediction(current,psf=current_retcf_pred)
#current_sepcf_pred = sepCF.predict_survival_function(current[sep_surv_cols1[2:]].values)

In [ ]:
ret_fiveyrDF = pd.concat([current[retire_surv_cols1[:3]],currentcfRetProb],axis=1)

In [ ]:
ret_fiveyrDF[['yr1','yr2','yr3','yr4','yr5']]=1-ret_fiveyrDF[['proba_1','proba_2','proba_3','proba_4','proba_5']]


In [ ]:
tmp_rates = pd.DataFrame(ret_fiveyrDF[['yr1','yr2','yr3','yr4','yr5']].sum()/float(len(ret_fiveyrDF)))
tmp_rates.index = ['yr1','yr2','yr3','yr4','yr5']
tmp_rates.columns = ['Retirement']
tmp_rates['Annual Retirement'] = tmp_rates.diff()
tmp_rates.loc['yr1','Annual Retirement']= tmp_rates.loc['yr1','Retirement']

In [ ]:
current_sepcf_pred = sepCF.predict_survival_function(current[sep_surv_cols1[2:]].values)
current_sepcf_pred.index.name='years'
current_sepcf_pred.reset_index(inplace=True)
current_sepcf_pred.head()
currentcfSepProb = get_survival_prediction(current,psf=current_sepcf_pred)
#current_sepcf_pred = sepCF.predict_survival_function(current[sep_surv_cols1[2:]].values)

In [ ]:
sep_fiveyrDF = pd.concat([current[sep_surv_cols1[:3]],currentcfSepProb],axis=1)


In [ ]:
sep_fiveyrDF[['Sep_1','Sep_2','Sep_3','Sep_4','Sep_5']]=1-sep_fiveyrDF[['proba_1','proba_2','proba_3','proba_4','proba_5']]


In [ ]:
sep_fiveyrDF.head()

In [ ]:
simple_rates = pd.DataFrame(sep_fiveyrDF[['Sep_1','Sep_2','Sep_3','Sep_4','Sep_5']].sum()/float(len(sep_fiveyrDF)))
simple_rates.index = ['yr1','yr2','yr3','yr4','yr5']
simple_rates.columns = ['Separation']
simple_rates['Annual Separation'] = simple_rates.diff()
simple_rates.loc['yr1','Annual Separation']= simple_rates.loc['yr1','Separation']


In [ ]:
simple_rates = pd.concat([simple_rates,tmp_rates],axis=1)
simple_rates

### Clean up missing & dummy encode/etc the categorical --> use a pipe

In [ ]:
missing_columns = br.get_columns_with_nulls(em2[all_cols3])
print len(missing_columns)

In [ ]:
missing_cat_cols = br.get_columns_with_nulls(em2[cat_cols2])
missing_cat_cols

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.base import BaseEstimator,TransformerMixin


In [ ]:
mthl_tenure_range = np.linspace(0,65,781)
def calculate_survival_functions_b(df,y, time_col, col_name,num_cutoff = 40,timerange =mthl_tenure_range):
    
    """ Function to generically return a dataframe of survival function, grouped by some categorical column
    inputs:
        df --> database to derive survival functions from
        time_col --> the temporal column to use for SF modeling (Kaplan Meier fitter applied)
        event_col --> the truncated column to use for SF modeling
        col_name --> the column to group up and determine KMF sf for
        num_cutoff --> number of groups to consider
        timerange --> min and max range
    outputs:
        survivalfunc_df --> a data frame that contains survival function.

    other options:
        *frac_cutoff --> the fraction of unique elements that will be kept as separate groups
        *min_size_cutoff --> min size to use for the cutoff.
        * these last two are not implemented
    """
    from lifelines import KaplanMeierFitter
    kmf=KaplanMeierFitter()
    # create example for all cases -- serves as background
    # create a time range
    
    kmf.fit(df[time_col],timeline=timerange,event_observed=y,label='all')
    survivalfunc_df = pd.DataFrame(kmf.survival_function_)
    # groupify the dataframe
    grp_value_counts = df[col_name].value_counts()
    #if frac_cutoff == None:
    #    #by default take 15 %
    #    frac_cutoff = .15 
    #top_n_groups = int(frac_cutoff *len(grp_value_counts))
    #if min_size_cutoff == None:
        # by default 
    # Take the top num_cutoff groups
    #my_grps = grp_value_counts.ix[:num_cutoff].index.tolist() 
    my_grps = grp_value_counts.iloc[:num_cutoff].index.tolist() 
    
    # make a list of elements in each of these groups
    grp_dict = {}
    for grp in my_grps: 
        grp_dict[grp] = df[df[col_name] == grp].index.tolist()
    # loop through grps and create kmf survival function
    for i,jgrp in enumerate(my_grps):
        j_idx = grp_dict[jgrp]
        #print i, jgrp, len(j_idx)
        kmf.fit(df[time_col].ix[j_idx],timeline=mthl_tenure_range,event_observed=y.ix[j_idx],label=str(jgrp))
        survivalfunc_df = pd.concat([survivalfunc_df,kmf.survival_function_],axis=1)
    
    
    return survivalfunc_df

def return_first_time_survival(sfdf,thresh=0.5):
    from collections import defaultdict
    
    # assign all value to the default
    default_value = sfdf[sfdf['all']<=thresh].index[0]
    median_survival_dict = defaultdict(lambda: default_value)
    for c in sfdf.columns[1:]:
        #print c
        try:
            my_sf_date = sfdf[sfdf[c]<=thresh].index[0]
        except IndexError: # because never reached that threshold value
            my_sf_date = sfdf.index[-1]
        except KeyError: # because of type of the key
            my_sf_date = sfdf[sfdf[int(c)]<=thresh].index[0]

        median_survival_dict[c]=my_sf_date
        
    return median_survival_dict

In [ ]:
class SurvivalEncodeColumn(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_fix=[],rows_to_scan='all',method='median',max_number_groups=40, my_thresh=0.5):
        self.method = method
        self.columns_to_fix = columns_to_fix
        self.rows_to_scan = rows_to_scan
        self.max_num_groups = max_number_groups
        self.thresh = my_thresh
        
    def fit(self,X,y):
        self.survive_columns = {} # dictionary to map old columns to new
        self.survive_values = {}
        self.column_cutoff = {}
        
        
        for col in self.columns_to_fix:
            #(assumes these have previously been label encoded so the values should be ints)"
            ncol=str(col)+"_le"
            num_cutoff = self.max_num_groups
            survive_col = 'surv_'+ncol.lower()
            nuniq = len(X[ncol].unique())
            if nuniq < num_cutoff:
                num_cutoff = nuniq
                
            self.column_cutoff[col]=num_cutoff
            
            #frac_accounted_for = X[col].value_counts().iloc[:num_cutoff].sum()/float(len(X))
            #print i,col,newcol, nuniq, num_cutoff,num_cutoff/float(nuniq),frac_accounted_for
            # I want to make this fraction close to 80%?
            sf_df = calculate_survival_functions_b(X,y,'Tenure_years', ncol,num_cutoff)
            self.survive_columns[col]=survive_col
            ## create the dictionary
            self.survive_values[col]=return_first_time_survival(sf_df,thresh=self.thresh)
    
        return self
    def transform(self,X,y=None):
        X_temp = X.copy()
        original_cols = list(X_temp.columns)
        for col in self.columns_to_fix:
            ncol=str(col)+"_le" # assumes labelencoded
            new_col = self.survive_columns[col]
            col_dict = self.survive_values[col]
            X_temp[new_col] = X_temp[ncol].apply(lambda x: col_dict[str(x)])
            original_cols.remove(ncol)
            original_cols.append(new_col)
        
        X_temp = X_temp[original_cols]
        return X_temp
    

In [ ]:
#my_cols_to_omit =list(set(all_cols)-set(all_cols3))
#len(my_cols_to_omit)

In [ ]:
[a for a in all_cols4 if a.endswith('SFI')]

In [ ]:
cat_cols2 = list(set(all_cols3).intersection(set(cat_cols2)))
[(c,len(em2[c].unique())) for c in cat_cols2]

In [ ]:
my_label_encode_cols = ['JOB_FAMILY', 'GRADE', 'JOB_FUNCTION',
 'FLOR_SFI', 'FUNC_ID_SFI', 'EXT_FUNC_ID_SFI',
 'JOBCODE', 'LOC_TYPE_DESCR_SFI',
 'LOC_STATE']
cat_cols_to_dummy_encode = ['EMPL_CLASS','EEO1CODE','EMPL_TYPE','FULL_PART_TIME','ETHNIC_GROUP','DIVISION_CODE_SFI','COMPANY']

In [ ]:
allcols = em2.columns.tolist()
my_cols_to_omit = ['TOT_MO_SERVICE_SFI']

In [ ]:
my_minfix_cols = ['SAL1','MIN_RT_ANNUAL','MERIT1','PERF1']
my_maxfix_cols = ['MAX_RT_ANNUAL']
missing_zero_cols = ['BOX1','FUNC_CNT','EXTFUNC_CNT','TOTAL_RPT_CNT','DIRECT_RPT_CNT']
mode_impute_cols = ['MERIT1','PERF1','ADDRCNT1']

In [ ]:
pipe2 = Pipeline([("null",br.RemoveAllNull()),
                 ("drop",br.DropColumns(columns_to_drop =my_cols_to_omit)),
                 ("label_encode",br.LabelEncodeColumn(my_label_encode_cols)),
                 
                 #("dummy_encode", br.DummyEncodeColumn(cat_cols_to_dummy_encode)),
                 ("survival_encode",SurvivalEncodeColumn(my_label_encode_cols[:5],method='median')),
                 ("dummy_encode",br.ConvertCategorical(categorical_columns=cat_cols_to_dummy_encode,method='dummy')),
                 ("fixout_min",br.FixNumericOutlier(columns_to_fix=my_minfix_cols,criteria_coef=('percentile',2),
                                                   method='lower',fill_with='nearest_value')),
                 ("fixout_max",br.FixNumericOutlier(columns_to_fix=['MAX_ANNUAL_RT'],criteria_coef=('percentile',2),
                                                   method='both',fill_with='nearest_value')),
                 ("fill_missingzero",br.FillMissingValue(columns_to_fix=missing_zero_cols,fill_value=0)),
                 ("imp_mode",br.ImputeData(columns_to_impute=mode_impute_cols,rows_to_scan=0.8))
                 ])

# pull out a build set to make the pipeline from

In [ ]:
print "Starting with subest of {0} employees.".format(len(em2))
eval_fraction = 0.20
em2build, em2eval = train_test_split(em2,test_size=eval_fraction,random_state=83221)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(em2eval),len(em2build))

In [ ]:
y = em2build['former']
build = pipe2.fit_transform(em2build,y)

In [ ]:
evaluation = pipe2.fit(em2build,y).transform(em2eval)

In [ ]:
list_of_columns = build.columns.tolist()

In [ ]:
[a for a in list_of_columns if a.endswith('SFI')]

In [ ]:
build.to_csv('trans_train.csv',index=False)
evaluation.to_csv('trans_eval.csv',index=False)

In [ ]:
print len(list_of_columns)
[a for a in list_of_columns if a in ['former','status','retired']]

In [ ]:
columns_for_modeling = [a for a in list_of_columns if a not in ['former','status','retired']]
len(columns_for_modeling)

In [ ]:
br.perfect_collinearity_test(build[columns_for_modeling])

In [ ]:
tstmp_cols = [a for a in em2.columns if a.endswith('tstmp')]
tstmp_cols

In [ ]:
redundant_cols = ['hire_age','FULLPART1','COMPANY','ETHNIC_GROUP',
                  'EMPL_TYPE','DIVISION_CODE_SFI','ADDRCNT1_d','PERF1_d','MERIT_d',
                  'EMPL_TYPE_E','FULL_PART_TIME_X','COMPANY_1','ETHNIC_GROUP_1.0','DIVISION_CODE_SFI_4',
                 'EMPL_CLASS_1','EEO1CODE_6']
columns_for_modeling2 = list(set(columns_for_modeling)-set(redundant_cols)-set(tstmp_cols))
br.perfect_collinearity_test(build[columns_for_modeling2])

In [ ]:
len(columns_for_modeling2)

# now build the time-kfolds

In [ ]:
# define the date range
full_date_range = [str(a)+'-01-01' for a in np.arange(2002,2016)]
print len(full_date_range)
#full_date_range

## Utilize temporal kfolds:
1. split the training dataset into windows corresponding to the difference in time
2. 


In [ ]:
import itertools
def create_temporal_kfolds(dates_df,date_range,time_delta):
    """
    inputs: date_range
            time_delta (in years)
            dataframe_dates --> assumes 'hire_tstmp' and 'term_tstmp'
    outputs:
            kf is a list of list of list: 
            [indices from original df that are 'in' a fold, indices from original df that are 'out' of a fold]
            each row in this corresponds to the temporal-kfold
            filtered_pairs is a list of start and end times
    """
    min_date = pd.to_datetime(date_range[0])
    max_date = pd.to_datetime(date_range[-1])
    my_index = dates_df[(dates_df.term_tstmp>=min_date)].index
    # calculate number of kfolds
    date_span_years = np.int(np.round((max_date-min_date).days/365.24,0))
    nfolds = date_span_years - time_delta
    print date_span_years, time_delta, nfolds, len(my_index)
    all_pairs = list(itertools.combinations(date_range,2))
    # now filter if difference in time  == time_delta
    filtered_pairs = []
    for i0,i1 in all_pairs:
        if int(i1[:4])-int(i0[:4]) == time_delta:
            filtered_pairs.append([i0,i1])
            #print i0,i1
    print len(filtered_pairs)
    # now process each of these filtered pairs
    kf = []
    for j0,j1 in filtered_pairs: # omit the last one because it has no corresponding partner/endtime
        start_date = pd.to_datetime(j0)
        end_date = pd.to_datetime(j1)
        #print j0,j1#,len(k)
        
        
        kfold_idx = dates_df[(dates_df.term_tstmp >= start_date) & (dates_df.hire_tstmp<start_date)].index.tolist()
        after_idx = dates_df[(dates_df.hire_tstmp>=end_date)].index.tolist()
        before_idx = list(set(my_index)-set(kfold_idx)-set(after_idx))
        #temporal_kfold(dates_df[dates_df.term_tstmp>=min_date],start_date,end_date)
        #print "\t",len(kfold_idx), len(after_idx),len(before_idx)
        
        # combined out of fold
        not_kfold_idx = list(set(after_idx).union(set(before_idx)))
        
        print j0,j1,len(kfold_idx),len(not_kfold_idx)
        kf.append([kfold_idx,not_kfold_idx])
    
    return kf,filtered_pairs

In [ ]:
def reset_years2(paired_times,indices,dates_df,cols_to_alter = ['Age_years','Tenure_years']):
    # calc the Age at beginnning of time period
    ## now calculate age at hire
    reset_age_tdelta = pd.to_datetime(paired_times[0])-dates_df['birth_tstmp']#)/np.timedelta64(1,'D')
    reset_tenure_tdelta = pd.to_datetime(paired_times[0])-dates_df['hire_tstmp']#)/np.timedelta64(1,'D')
    # convert to days, months or years
    reset_age = reset_age_tdelta/np.timedelta64(1,'Y')
    reset_tenure = reset_tenure_tdelta/np.timedelta64(1,'Y')
    # look at terminated or not
    #empl_df['terminated']= 0
    # push these as a data frame so I can merge
    adj_times = pd.DataFrame()
    adj_times['adj_age']=reset_age
    adj_times['adj_tenure']=reset_tenure      
    return adj_times


In [ ]:
def define_target_within_x_years(df,paired_dates,tfold,n_years,target_col):
    """
    inputs:
        df -- data frame
        paired_dates (2nd output from create_temporal_kfolds)
        tfold (1st output from create_temporal_kfolds)
        targe_col --> desired target
        
    plan is to adjust the target, Age and tenure to match time frame of temporal kfold
    outputs:
    """
    print len(tfold)#, paired_dates

    df_dict = {}
    for i,tf in enumerate(tfold):
        start_date = paired_dates[i][0]
        end_date = paired_dates[i][1]
        #print start_date,end_date,n_years
        #altered_fold_df = pd.DataFrame(columns=['fold_mbr','adj_age','adj_tenure','adj_term'])
        # if in the fold reset the age to start of fold; define new window of termination
        in_fold_idx = tfold[i][0] # the index of the in_fold_observations 
        # note that "sex" is just used to create a value that then gets dummied out
        cols_to_copy = ['SEX','Age_years','Tenure_years']
        cols_to_copy.append(target_col)
        
        altered_fold_df = df[cols_to_copy].copy()
        # adjust these
        altered_fold_df.columns=['fold_mbr','adj_age','adj_tenure','adj_tgt']
        altered_fold_df.fold_mbr = 0
        
        #ra,rt = reset_years(paired_dates[i],in_fold_idx,dates_df)
        adj_df =reset_years2(paired_dates[i],in_fold_idx,df)
        #altered_fold_df.ix[in_fold_idx]['adj_age']=ra
        #altered_fold_df.ix[in_fold_idx]['adj_tenure']=rt
        altered_fold_df.loc[in_fold_idx,['adj_age','adj_tenure']]=adj_df.ix[in_fold_idx] # assign to the altered values
        new_tgt = (df.ix[in_fold_idx]['term_tstmp']<= end_date).as_matrix().astype(np.int)
        #(em2mod.ix[kf[0][0]]['term_tstmp']<= fp[0][1]).as_matrix().astype(np.int)
        # deal with last time-fold specially
        if i == len(tfold)-1:
            new_tgt = (df.ix[in_fold_idx]['term_tstmp']< end_date).as_matrix().astype(np.int)
        #print "\t", len(new_term),sum(new_term)
        altered_fold_df.loc[in_fold_idx,'adj_tgt']=new_tgt
        altered_fold_df.loc[in_fold_idx,'fold_mbr']=1
        df_dict[i]=altered_fold_df
    # now append this to a larger panel
    tfold_panel = pd.Panel.from_dict(data =df_dict)
    return tfold_panel

In [ ]:
def setup_tfolds(df, n_years, cols_to_model, tgt_value='former',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = tstmp_cols, date_range=full_date_range):
    """ Function to create the temporal kfolds
    parameters:
        df       - a pandas data frame (that has been preprocessed)
        n_years  - size of the temporal window (in years)
        cols_to_model - list of columns to use in the modeling procedure 
        tgt_value  - column name to use as the target variable
        time_cols  - list of columns that are time senitive and must be adjusted
        tstmp_cols - list of columns that contain timestamps for things like hiring, termination and birth
        date_range - list comprising the inclusive annual starting dates from beginning to end of range.
    output:
        tfolds  - a list containing the indices of in and out of fold members for each of the tfolds created
        tfold_panel - a pandas Panel (3D data frame) containing the membership in different folds, the adjusted targets,
            the adjusted timesenitive data, and the columns to use for modeling.
    """    
    # step 1 create the temporal_folds
    if not [a for a in tstmp_cols if a in df.columns] == tstmp_cols:
        print "The list of required timestamp columns do not exist in the input dataframe."
        print tstmp_cols
        exit
    tfolds, paired_dates = create_temporal_kfolds(df[tstmp_cols],date_range, n_years) 
    #defines the time-windows and assigns observations to folds
    
    # create the time-shifted tfolds and push into a pandas Panel (3D data frame)
    tfold_panel=transform_df_to_tfold(df,tfolds,paired_dates,n_years,cols_to_model, tgt_value,tstmp_cols,time_cols)
    return tfold_panel, tfolds
   

In [ ]:
def transform_df_to_tfold(df,tfold,paired_dates,n_years,cols_to_keep,target_col,tstmp_cols,time_cols):
    """
    Function to adjust the input data frame so that time_sensitive columns and target column
    conform to the bounds of that temporal kfold (tfold). Specifically adjust the target, Age and tenure 
    to match time frame of temporal kfold
    
    Parameters:
        df -- a pandas data frame
        paired_dates (2nd output from create_temporal_kfolds)
        tfolds (1st output from create_temporal_kfolds)
        target_col --> desired target
        cols_to_keep --> the list of columns within df that get expressed in df.
        time_cols
        
    plan is to 
    outputs:
        tfold_panel
    """
    #print len(tfold)#, paired_dates
    #print df.columns ,len(df.columns), len(cols_to_keep)
    #return
    # do some sanity checks
    # 1. make sure cols_to_keep, target and tstmp_cols columns are in df.columns
    input_column_list = df.columns.tolist()
    n_total_columns = len(input_column_list)
    #if (n_total_columns != len(cols_to_keep)+len(target_col)+len(tstmp_cols)):
        # now look at each piece
    targetlist = []
    targetlist.append(target_col)
    if not check_if_subset(targetlist,input_column_list):
            #target_col in df.columns:
            #print "{0} target column not present in input set".format(target_col)
        return
    if not check_if_subset(tstmp_cols,input_column_list):
        return
            #[a for a in tstmp_cols if a in input_column_list] == tstmp_cols: # requires
            #print "There are missing timestamp columns. Recheck the list {0}".format(tstmp_cols)
    #        return
    if not check_if_subset(time_cols,input_column_list):
            #[a for a in time_cols if a in input_column_list] == time_cols: # requires
            #print "There are missing timestamp columns. Recheck the list {0}".format(tstmp_cols)
        return    
    if not check_if_subset(cols_to_keep,input_column_list):
        return
    
    ## now set up the columns for manipulation
    cols_to_copy = cols_to_keep
    if not check_if_subset(time_cols,cols_to_keep):
        cols_to_copy+=time_cols
        print len(cols_to_copy)
    
    
    df_dict = {}
    
    # identify which element of tstmp_cols corresponds to term_tstmp
    term_id = tstmp_cols.index('term_tstmp')
    birth_id = tstmp_cols.index('birth_tstmp')
    hire_id = tstmp_cols.index('hire_tstmp')
    #print term_id, tstmp_cols[term_id]
    
    for i,tf in enumerate(tfold):
        start_date = paired_dates[i][0]
        end_date = paired_dates[i][1]
        
        in_fold_idx = tfold[i][0] # the index of the in_fold_observations 
       
        altered_fold_df = df[cols_to_copy].copy()
        altered_fold_df['fold_mbr']=0 # create a new column to identify membership in that fold.
        altered_fold_df.loc[in_fold_idx,'fold_mbr']=1 # assign the in_fold_indices to that fold membership
        ####
        # adjust the age and tenure for those in the fold
        reset_age_tdelta = pd.to_datetime(start_date)-df.ix[in_fold_idx][tstmp_cols[birth_id]]
        reset_age = reset_age_tdelta/np.timedelta64(1,'Y')
        reset_tenure_tdelta = pd.to_datetime(start_date)-df.ix[in_fold_idx][tstmp_cols[hire_id]]
        reset_tenure = reset_tenure_tdelta/np.timedelta64(1,'Y')
        altered_fold_df.loc[in_fold_idx,'Age_years']=reset_age
        altered_fold_df.loc[in_fold_idx,'Tenure_years']=reset_tenure
        old_tgt = df.ix[tfold[i][1]][target_col]
        new_tgt = (df.ix[in_fold_idx][tstmp_cols[term_id]]<= end_date).as_matrix().astype(np.int)
        # deal with last time-fold specially
        if i == len(tfold)-1:
            new_tgt = (df.ix[in_fold_idx][tstmp_cols[term_id]]< end_date).as_matrix().astype(np.int)
            
        altered_fold_df.loc[in_fold_idx,target_col]=new_tgt
        old_tgt = df.ix[tfold[i][1]][target_col]
        altered_fold_df.loc[tfold[i][1],target_col] = old_tgt
        #ra,rt = reset_years(paired_dates[i],in_fold_idx,dates_df)
        """adj_df =reset_years2(paired_dates[i],in_fold_idx,df)
        #altered_fold_df.ix[in_fold_idx]['adj_age']=ra
        #altered_fold_df.ix[in_fold_idx]['adj_tenure']=rt
        altered_fold_df.loc[in_fold_idx,time_cols]=adj_df.ix[in_fold_idx] # assign to the altered values
        
        #(em2mod.ix[kf[0][0]]['term_tstmp']<= fp[0][1]).as_matrix().astype(np.int)
        
        #print "\t", len(new_term),sum(new_term)
        
        """
        
        df_dict[i]=altered_fold_df
    # now append this to a larger panel
    tfold_panel = pd.Panel.from_dict(data =df_dict)
    return tfold_panel
    
    
    

In [ ]:
def apply_tKfold_CV2(modeltype, in_panel, tkfolds, cols_to_use, tgt_column = 'former', ntrees=100):
    """ Calculate the rmse for each cross-validated temporalFold
    
    Parameters:
        model - a scikit learn model name
        in_panel  - a pandas Panel with the observed input data adjusted 
        tgt_column -- the target variable
        cols_to_use -- the input variables to be used in modeling
        tkfolds -- teh temporal timefolds (list of dimension 2)
    outputs:
        models --> the list of fit models
        RMSE --> the rmse error
        
    """
    RMSE = []
    roc_auc = []
    models = []
    
    
    
    for fold_id, indices in enumerate(tkfolds):
        training = indices[0]
        testing = indices[1]
        #print fold_id, len(training),len(testing)
        X_train, X_test = in_panel[fold_id][cols_to_use].ix[training].as_matrix(), in_panel[fold_id][cols_to_use].ix[testing].as_matrix()
        y_train, y_test = in_panel[fold_id][tgt_column].ix[training].as_matrix(), in_panel[fold_id][tgt_column].ix[testing].as_matrix()
        if modeltype == 'rfc':
            model = ensemble.RandomForestClassifier(n_estimators=ntrees, max_features='auto', oob_score=True)
        elif modeltype=='bgc':
            model=ensemble.GradientBoostingClassifier(n_estimators=ntrees,max_features='auto')
        else:
            model=tree.DecisionTreeClassifier()
            
        # Train the model
        model.fit(X_train,y_train)
        # use the model to predict output
        y_fitted = model.predict(X_test)
        RMSE.append(np.sqrt(metrics.mean_squared_error(y_test,y_fitted)))
        roc_auc.append(metrics.roc_auc_score(y_test,y_fitted))
        models.append(model)
    # leave the model fit to the entire dataset
    #model.fit(in_x,in_y)
    
    return RMSE,roc_auc,models
        

In [ ]:
def check_if_subset(list1,list2): #compare_column_lists(list1,list2):
    if set(list1).issubset(set(list2)):
        return 1
    else:
        print "{0} is not contained in {1}. Recheck your lists.".format(list1,list2)
        return 0
    
    

## reintroduce the tstmp columns

In [ ]:
build[tstmp_cols]=em2build[tstmp_cols].apply(lambda x:  pd.to_datetime(x))

In [ ]:
panel4, tfold4 = setup_tfolds(build, 4, columns_for_modeling2, tgt_value = 'former',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols=tstmp_cols, date_range=full_date_range)
   

# Now  apply gbc

In [ ]:
from sklearn import ensemble, tree, metrics

In [ ]:
rmse4,roc_auc4,gbmdl4 = apply_tKfold_CV2('gbc',panel4, tfold4,  cols_to_use = columns_for_modeling2, tgt_column='former',ntrees=500)#['columns_for_modeling'],em2mod['former'],)

In [ ]:
gbc_eval_pred4class, gbc_eval_pred4proba = evaluate_models(gbmdl4,X4eval)

In [ ]:
plot_roc_curve(y4eval,gbc_eval_pred4proba[:,:,:].mean(axis=2))

In [ ]:
plot_conf_matrix(y4eval,map(np.int,gbc_eval_pred4class.mean(axis=1)))

## build rf model

In [ ]:
rmse4rf,roc_auc4rf,rfmdl4 = apply_tKfold_CV2('rfc',panel4, tfold4,  cols_to_use = columns_for_modeling2, tgt_column='former',ntrees=100)#['columns_for_modeling'],em2mod['former'],)

In [ ]:
rfc_eval_pred4class, rfc_eval_pred4proba = evaluate_models(rfmdl4,X4eval)

In [ ]:
evaluation[tstmp_cols]=em2eval[tstmp_cols].apply(lambda x: pd.to_datetime(x)) # append the tstmp files

In [ ]:
X4eval, y4eval =adjust_eval_by_x_years(evaluation,4,columns_for_modeling2)#,tstmp_cols=default_tstmps,target_col='former')

In [ ]:
rmse4, rmse4rf

In [ ]:
roc_auc4, roc_auc4rf

In [ ]:
#plot_roc_curve(y4eval,np.vstack([gbc_eval_pred4class.mean(axis=1), gbc_eval_pred4class.mean(axis=1)]).T)

In [ ]:
plot_roc_curve(y4eval,rfc_eval_pred4proba[:,:,:].mean(axis=2))

#REPEAT for retirements

In [ ]:
Rpanel4, Rtfold4 = setup_tfolds(evaluation,4, columns_for_modeling2, tgt_value = 'retired',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
Rrmse4,Rroc_auc4,Rrfmdl4 = apply_tKfold_CV2('rfc',Rpanel4, Rtfold4, cols_to_use = columns_for_modeling2, tgt_column='retired')#['columns_for_modeling'],em2mod['former'],)


In [ ]:
#float(
((today - evaluation[default_tstmps[2]].iloc[0]).days)/365.25

In [ ]:
RX4eval, Ry4eval = adjust_eval_by_x_yearsR(evaluation,4,columns_for_modeling2, today,target_col = 'retired')
Reval_pred4class,Reval_pred4proba = evaluate_models(Rrfmdl4,RX4eval)
plot_roc_curve(Ry4eval,Reval_pred4proba[:,:,:].mean(axis=2))

## retry retirements using undersampling

In [ ]:
from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids
#OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

#### count the initial classes

In [ ]:
# 'Random under-sampling'
US = UnderSampler(ratio=2)

In [ ]:
Rpanel4, Rtfold4 = setup_tfolds(evaluation,4, columns_for_modeling2, tgt_value = 'retired',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)

In [ ]:
print len(Rpanel4)
len(Rpanel4[0])

In [ ]:
Rpanel4[0].retired.sum()

In [ ]:
Rpanel_cols = Rpanel4[0].columns.tolist()
Rpanel_cols.remove('retired')


In [ ]:
usx,usy = US.fit_transform(Rpanel4[0][Rpanel_cols].values, Rpanel4[0]['retired'].values)

In [ ]:
usRpanel4[0] = pd.DataFrame(data=usx,columns=Rpanel_cols)
usRpanel4[0]['retired']= usy


In [ ]:
for i in xrange(0,len(Rtfold4)):
    print i, len(Rtfold4[i][0]), len(Rtfold4[i][1])

In [ ]:
Rpanel4[0].ix[Rtfold4[0][1]].retired.value_counts()

In [ ]:
def apply_tKfold_CV2_unBalanced(modeltype, in_panel, tkfolds, cols_to_use, unbalance_mtd='random', tgt_column = 'former', ntrees=100):
    """ Calculate the rmse for each cross-validated temporalFold
    
    Parameters:
        model - a scikit learn model name
        in_panel  - a pandas Panel with the observed input data adjusted 
        tgt_column -- the target variable
        cols_to_use -- the input variables to be used in modeling
        tkfolds -- the temporal timefolds (list of dimension 2)
        unbalance_mtd -- a method of how to deal with imbalanced classes
    outputs:
        models --> the list of fit models
        RMSE --> the rmse error
        
    """
    RMSE = []
    roc_auc = []
    models = []
    
    
    
    for fold_id, indices in enumerate(tkfolds):
        training = indices[0]
        testing = indices[1]
        
        # 'Random under-sampling'
        if unbalance_mtd == 'random':
            sampler = UnderSampler(ratio=2)
        else:
            sampler = NearMiss(version=3)

        
        #print fold_id, len(training),len(testing)
        X_train, X_test = in_panel[fold_id][cols_to_use].ix[training].as_matrix(), in_panel[fold_id][cols_to_use].ix[testing].as_matrix()
        y_train, y_test = in_panel[fold_id][tgt_column].ix[training].as_matrix(), in_panel[fold_id][tgt_column].ix[testing].as_matrix()
        usXtrain, usytrain = sampler.fit_transform(X_train,y_train)
        
        if modeltype == 'rfc':
            model = ensemble.RandomForestClassifier(n_estimators=ntrees, max_features='auto', oob_score=True)
        elif modeltype=='bgc':
            model=ensemble.GradientBoostingClassifier(n_estimators=ntrees,max_features='auto')
        else:
            model=tree.DecisionTreeClassifier()
            
        # Train the model
        print len(usXtrain)
        model.fit(usXtrain,usytrain)
        
        # use the model to predict output
        y_fitted = model.predict(X_test)
        RMSE.append(np.sqrt(metrics.mean_squared_error(y_test,y_fitted)))
        roc_auc.append(metrics.roc_auc_score(y_test,y_fitted))
        models.append(model)
    # leave the model fit to the entire dataset
    #model.fit(in_x,in_y)
    
    return RMSE,roc_auc,models
        

In [ ]:
uRrmse4,uRroc_auc4,uRrfmdl4 = apply_tKfold_CV2_unBalanced('rfc',Rpanel4, Rtfold4, cols_to_use = columns_for_modeling2, tgt_column='retired')

In [ ]:
#plt.scatter(uRroc_auc4, Rroc_auc4)
plt.plot(uRroc_auc4)

In [ ]:
def evaluate_models(model_list,Xeval):
    eval_pred_class = np.zeros((len(Xeval),len(model_list)))
    eval_pred_proba = np.zeros((len(Xeval),2,len(model_list)))

    for i,mdl in enumerate(model_list):
        eval_proba = mdl.predict_proba(Xeval)
        eval_pred_class[:,i]=mdl.predict(Xeval)
        eval_pred_proba[:,:,i]=eval_proba
    #print np.shape(eval_prediction_proba3)
    return eval_pred_class, eval_pred_proba

In [ ]:
for a in xrange(0,len(uRrfmdl4)):
    plot_roc_curve(Ry4eval,uReval_pred4proba[:,:,a])

In [ ]:
plot_roc_curve(Ry4eval,uReval_pred4proba[:,:,9])

In [ ]:
uReval_pred4class,uReval_pred4proba = evaluate_models(uRrfmdl4,RX4eval)
plot_roc_curve(Ry4eval,uReval_pred4proba[:,:,:].mean(axis=2))

In [ ]:
print len(uReval_pred4proba[:,0,9])
#len(uReval_pred4class)
uReval_pred4class[:,8].sum(), Ry4eval.sum()

In [ ]:
uReval_pred4proba[:,:,4]

In [ ]:
plot_conf_matrix(Ry4eval,uReval_pred4class[:,7])

In [ ]:
#plt.plot(Ry4eval,'o')
#plt.plot(uReval_pred4class[:,2],'d')
#plt.scatter(Ry4eval,Ry4eval-uReval_pred4class[:,2])


In [ ]:

#zip(columns_for_modeling2, gbmdl4[0].feature_importances_)

In [ ]:
[columns_for_modeling2[a] for a in list(np.where(gbmdl4[1].feature_importances_ > 0.1)[0])]

In [ ]:
for i in xrange(0,len(gbmdl4)):
    plt.plot(gbmdl4[i].feature_importances_)
    
#plt.plot(gbmdl4[1].feature_importances_)

In [ ]:
X5eval, y5eval = adjust_eval_by_x_years(evaluate,5,columns_for_modeling2)

In [ ]:
em2[em2.retired==1].Age_years.min()

In [ ]:
em2[em2.retired==1].Age_years.hist(bins=40)

In [ ]:
len(em2[(em2.retired == 1) & (em2.Age_years < 55)]), len(em2[em2.retired == 1])

In [ ]:
em2[(em2.retired == 1) & (em2.Age_years < 55)].Age_years.hist(bins=30)

In [ ]:
#save the over50 set
over50.to_csv('over50.csv',index=False)

In [ ]:
over50.isnull().any()

In [ ]:
from sklearn import cross_validation

In [ ]:
# repeat the split

# break into evaluation and build sets
print "Starting with subset of {0} employees.".format(len(over50))
eval_fraction = 0.20
o50build, o50eval = cross_validation.train_test_split(over50,test_size=eval_fraction,random_state = 84)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(o50eval),len(o50build))

In [ ]:
print over50.retired.value_counts()
print "----"
print o50eval.retired.value_counts()
print "----"
print o50build.retired.value_counts()
print "----"

In [ ]:
o50build['JOBCODE'].head()

In [ ]:
o50build.columns

In [ ]:
retire_surv_cols1 = ['retired','Age_years','hire_age','SEX','FTE','HAVE_INS','HAVE_DEP']
cf.fit(o50build[retire_surv_cols1], 'Age_years', event_col='retired')

cf.print_summary()

In [ ]:
cf.hazards_

In [ ]:
cf.baseline_cumulative_hazard_.plot()

In [ ]:
first10pred = cf.predict_survival_function(o50eval[retire_surv_cols1[2:]].iloc[:10].values)

In [ ]:
cf_pred = cf.predict_survival_function(o50eval[retire_surv_cols1[2:]].values)


In [ ]:
first10pred[1].plot()

In [ ]:
o50eval[retire_surv_cols1[:2]].head(10)

In [ ]:
[first10pred[first10pred[a]<0.5].index[0] for a in xrange(0,10)]

In [ ]:
from lifelines import AalenAdditiveFitter
aaf = AalenAdditiveFitter(coef_penalizer=1.0, fit_intercept=True)

In [ ]:
aaf.fit(o50build[retire_surv_cols1], 'Age_years', event_col='retired')


In [ ]:
aaf.plot( columns=[ 'hire_age', 'baseline', 'SEX','FTE' ],ix=slice(53,70)) # cumulative hazard

In [ ]:
aaf.cumulative_hazards_.ix[:70].plot()

In [ ]:
aaf.predict_survival_function(o50eval[retire_surv_cols1[2:]].iloc[:3].values).plot()

In [ ]:
aaf.predict_survival_function(o50eval[retire_surv_cols1[2:]].iloc[:10].values).ix[:70].plot()

## how to apply this
1. take current age, add 1 yr, 2, yr, 3, yr, ,4yr, 5yr and report prob

In [ ]:
o50eval[retire_surv_cols1].head()

In [ ]:
#ages_to_check = o50eval['Age_years'].apply(lambda x: x+[1.0,2.0,3.0,4.0,5.0])

In [ ]:
pred_sf1 = aaf.predict_survival_function(o50eval[retire_surv_cols1[2:]].values)
# columns are the indiviuals.

In [ ]:
pred_sf1.index.name='years'
pred_sf1.reset_index(inplace=True)

In [ ]:
cf_pred = cf.predict_survival_function(o50eval[retire_surv_cols1[2:]].values)
cf_pred.index.name='years'
cf_pred.reset_index(inplace=True)

In [ ]:
len(pred_sf1.T), len(o50eval)

In [ ]:
#cfRetProb.head(), retireProb_df.head()

In [ ]:
retireProb_df = get_survival_prediction(o50eval)

In [ ]:
pred_o50eval=pd.concat([o50eval[retire_surv_cols1],retireProb_df],axis=1)
pred_o50eval_cf = pd.concat([o50eval[retire_surv_cols1],cfRetProb],axis=1)

In [ ]:
pred_o50eval.head()

## try to assess current age for retired or not

In [ ]:
o50eval_curr.Age_years.hist()#tail()#proba_0.describe()


In [ ]:
o50eval_curr[o50eval_curr.retired==1].proba_0.hist(bins=30,label='retired',normed=True,alpha=0.8)
print sum(o50eval_curr.retired==1), sum(o50eval_curr.retired==0)
o50eval_curr[o50eval_curr.retired==0].proba_0.hist(bins=30,label='active',normed=True,alpha=0.3)
plt.legend()
plt.xlabel('Retirement Probability')
plt.ylabel('Normed Counts')

In [ ]:
CFcurr_retire_age = get_survival_prediction(o50eval,psf=cf_pred,yr_vals=[0])
o50eval_curr_cf = pd.concat([o50eval[retire_surv_cols1],CFcurr_retire_age],axis=1)

## assess the model
* define a cutoff in probability and do a count/assessment

In [ ]:
def get_accuracy(df,true_col = 'retired', p_col = 'proba_0',thresh=0.5):
    ckdf = (df[p_col]>thresh).astype('int')
    true_pos = df[ckdf==1][true_col].sum()
    false_pos= len(df[ckdf==1])-true_pos
    
    false_neg=df[ckdf==0][true_col].sum()
    true_neg =len(df[ckdf==0]) - false_neg
    print "threshold = {0}".format(thresh)
    print "TP\t FP\t FN\t TN"
    print true_pos, '\t',false_pos,'\t' ,false_neg,'\t', true_neg
    acc = (true_pos + true_neg)/float(len(df))
    print "Accuracy is {0:.3f}".format(acc)
    mcc = (true_pos*true_neg - false_pos*false_neg)/np.sqrt((true_pos+false_pos)*(true_pos+false_neg)*(true_neg+false_pos)*(true_neg+false_neg))
    print "Matthews correlation coefficient is {0:.3f}".format(mcc)
    tpr = true_pos/float(true_pos+false_neg)
    fpr = false_pos/float(false_pos + true_neg)
    return tpr,fpr

In [ ]:
get_accuracy(o50eval_curr,thresh=0.5)

In [ ]:
roc_vals = []
tvals = np.linspace(0,1,21)#,0.05)#:#xrange(0,1,0.05):
for t in tvals:
    roc_vals.append(get_accuracy(o50eval_curr,thresh=t))

    
roc_vals = np.array(roc_vals)
#    print i

In [ ]:
roc_vals = np.array(roc_vals)
plt.plot(roc_vals[:,0],roc_vals[:,1])
plt.plot(tvals,tvals,color='k',ls='--')
plt.xlabel('FPR')
plt.ylabel('TPR')

In [ ]:
metrics.auc(roc_vals[:,0],roc_vals[:,1])

In [ ]:
roc_vals_cf = []
tvals = np.linspace(0,1,21)#,0.05)#:#xrange(0,1,0.05):
for t in tvals:
    roc_vals_cf.append(get_accuracy(o50eval_curr_cf,thresh=t))

    
roc_vals_cf = np.array(roc_vals_cf)

plt.plot(roc_vals_cf[:,1],roc_vals_cf[:,0])
plt.plot(tvals,tvals,color='k',ls='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('CoxPH Fitter of eval set')
print metrics.auc(roc_vals_cf[:,0],roc_vals_cf[:,1])

In [ ]:
plt.plot(roc_vals_cf[:,0],roc_vals_cf[:,1])
plt.plot(roc_vals[:,1],roc_vals[:,0])

In [ ]:
c02 = (o50eval_curr.proba_0 > 0.2).astype('int')
sum(c02 == o50eval_curr.retired), len(o50eval_curr)

In [ ]:
o50eval_curr[c02==1].retired.sum(), len(o50eval_curr[c02==1]), o50eval_cur[c02==0].retired.sum()

In [ ]:
o50eval[retire_surv_cols1].head(10)

In [ ]:
cf_med_survAge = cf.predict(o50eval[retire_surv_cols1[2:]])


In [ ]:
cf_med_survAge.head()

In [ ]:
cf.predict_expectation(o50eval[retire_surv_cols1[2:]].head(10))

## apply this model to all current to get workforce retirement predictions

In [ ]:
pred_current = aaf.predict_survival_function(all_current[retire_surv_cols1[2:]].values)

In [ ]:
pred_current.head()
pred_current.index.name='years'
pred_current.reset_index(inplace=True)

In [ ]:
pred_current[['years',0,1]]

In [ ]:
all_current[retire_surv_cols1].head()

In [ ]:
all_current[retire_surv_cols1].ix[65080:65093]

In [ ]:
current_retireProb_df = get_survival_prediction(all_current[retire_surv_cols1],psf=pred_current)

In [ ]:
current_retireProb_df.head()

## Create a set that is under 50 for separation modeling


In [ ]:
under50 = em2[em2.Age_years <= 50].copy()
print len(under50)

In [ ]:
under50.HAVE_DEP.value_counts()

In [ ]:
sep_surv_cols1=['former','Tenure_years','hire_age','SEX','FTE','HAVE_INS','HAVE_DEP','SAL1']

In [ ]:

# break into evaluation and build sets
print "Starting with subset of {0} employees.".format(len(under50))
eval_fraction = 0.20
u50build, u50eval = cross_validation.train_test_split(under50,test_size=eval_fraction,random_state = 84554)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(u50eval),len(u50build))

In [ ]:
br.perfect_collinearity_test(u50build[sep_surv_cols1])

In [ ]:
cf.fit(u50build[sep_surv_cols1], 'Tenure_years', event_col='former')

cf.print_summary()



In [ ]:
cf.hazards_

In [ ]:
sep_cf_pred = cf.predict_survival_function(u50eval[sep_surv_cols1])

## Now transform this panel into an h2o Frame

In [ ]:
len(panel4), len(tfold4)

#### one off 
* for each mdl in len(panel4):
    * train = panel4[mdl].ix[tfold4[mdl][0]]
    * test = panel4[mdl].ix[tfold4[mdl][1]]

In [ ]:
my_train_df = panel4[0].ix[tfold4[0][0]]
my_test_df =  panel4[0].ix[tfold4[0][1]]
#train4_0 = h2o.H2OFrame(panel4[0][columns_for_modeling2].ix[tfold4[0][0]])
#y4_0 = h2o.H2OFrame(python_obj=panel4[0]['former'].ix[tfold4[0][0]])
my_train_df.shape, my_test_df.shape

In [ ]:
my_train_df[].shape

In [ ]:
h2o_cols = my_train_df.columns.tolist()
h2o_cols.remove('fold_mbr')

In [ ]:
!mkdir tmp
os.chdir('tmp')

In [ ]:
my_train_df[h2o_cols].to_csv('my_train.csv',index=False)
my_test_df[h2o_cols].to_csv('my_test.csv',index=False)

In [ ]:
# dictionary does not preserve the colum order/data
#train4_0 = h2o.H2OFrame(python_obj = my_train_df[h2o_cols].to_dict())
#test4_0 = h2o.H2OFrame(python_obj = my_test_df[h2o_cols].to_dict())
train4_0 = h2o.H2OFrame?

In [ ]:
!hdfs dfs -put *.csv /user/kesj/ajH2O_3

In [ ]:
train4_0 = h2o.import_file(path = 'hdfs://nameservice1/user/kesj/ajH2O_3/my_train.csv')
test4_0 = h2o.import_file(path = 'hdfs://nameservice1/user/kesj/ajH2O_3/my_test.csv')

In [ ]:
h2o_gbm_mdl_0 = h2o.gbm(y = "former", x = columns_for_modeling2, training_frame = train4_0, validation_frame = test4_0, ntrees = 500 , max_depth = 4, learn_rate=0.1,distribution="AUTO")

In [ ]:
h2o_gbm_mdl_0.deepfeatures

In [ ]:
gbm_0_vi = pd.DataFrame(data =h2o_gbm_mdl_0.varimp(return_list=True),columns=['variable','relative_importance','scaled_importance','percentage'])
gbm_0_vi

In [ ]:
gbm_0_vi.percentage.cumsum().plot()

## predict on the hold out set


In [ ]:
default_tstmps = ['birth_tstmp','hire_tstmp','term_tstmp']

In [ ]:
def adjust_eval_by_x_years(df,year_val,modeling_columns,tstmp_cols=default_tstmps,target_col='former'):
    # construct
  
    ## set up method to assess the eval set
    print "There are {0} elements in the evaluation set".format(len(df))
   
    print "original target variable value counts:", df[target_col].value_counts()
    # restructure to deal with time_frame retirement (target variable)
    yr_cut_val = year_val+0.5
    # index of those that actually accomplish target within timeframe (allow 0.5 additional years)
    eval_within_time_target_index = df[(df[target_col]==1) & (df.Tenure_years <= yr_cut_val)].index
    # exclude indices that are active and have tenure less than this time
    eval_excluded_index = df[(df[target_col]==0) & (df.Tenure_years  <= yr_cut_val)].index
    
    # the rest become my not-terminated set
    eval_active_index = set(df.index) - set(eval_within_time_target_index) - set(eval_excluded_index)
    print len(eval_excluded_index),len(eval_within_time_target_index), len(eval_active_index)
    eval_idx_to_use =df.ix[set(df.index)-set(eval_excluded_index)].index
    #len(eval_idx_to_use)
    # reset the target to 0 for active
    eval_new_target = df[target_col].copy()
    eval_new_target.ix[eval_active_index] = 0
    print "new target variable value counts: "
    print eval_new_target.ix[eval_idx_to_use].value_counts()
    print "_____"
    y_eval = eval_new_target.ix[eval_idx_to_use].as_matrix().astype(np.int) # true values
    eval_adj_tenure = df.ix[eval_idx_to_use].Tenure_years.apply(lambda x: x-year_val if (x>float(year_val)) else 0).values
    print len(eval_adj_tenure), len(y_eval)
    # now adjust age by length of time; use hire_age if not in set to use.
    eval_adj_age = df.ix[eval_idx_to_use].Age_years.apply(lambda x: x-year_val)
    hire_age = (df.ix[eval_within_time_target_index]['birth_tstmp']-df.ix[eval_within_time_target_index]['hire_tstmp'])/np.timedelta64(1,'Y')#.days/days_in_year
    eval_adj_age.ix[eval_within_time_target_index] = hire_age #df.ix[eval_within_time_target_index]['birth_tstmp'#df_dates['hire_age']
    
    # construct the evaluation X matrix
    print "input matrix has {0} features".format(len(modeling_columns))
    Xeval = np.zeros((len(eval_idx_to_use),len(modeling_columns)))
    # drop 'Age_years' and Tenure_years from the list
    cols_to_use = []
    cols_to_use+=modeling_columns
    cols_to_use.remove('Age_years')
    cols_to_use.remove('Tenure_years')#.copy()
    """
    Xeval[:,:-2] = df.ix[eval_idx_to_use][cols_to_use].as_matrix().astype(np.float)
    # now put the adjusted tenure and ages into this matrix
    Xeval[:,-2] = eval_adj_age.values
    Xeval[:,-1]=eval_adj_tenure
    #print len(modeling_columns),np.shape(Xeval)
    """
    # this version matches the changed ording of columns
    Xeval[:,0]=eval_adj_age.values
    Xeval[:,1]=eval_adj_tenure
    Xeval[:,2:] = df.ix[eval_idx_to_use][cols_to_use].as_matrix().astype(np.float)
    return Xeval, y_eval

In [ ]:
eval_dict = {}
for idx,feature in enumerate(columns_for_modeling2):
    print idx, feature
    eval_dict[feature]=list(X4eval[:,idx])
    
eval_dict['former']=list(y4eval)

In [ ]:
eval4_frame = h2o.H2OFrame(python_obj=eval_dict)

#h2o_gbm_mdl_0.predict()

In [ ]:
eval4_frame.as_data_frame().to_csv('eval_4.csv',index=False)

In [ ]:
['former' in eval4_frame.columns]

In [ ]:
pred4_0 = h2o_gbm_mdl_0.predict(eval4_frame).as_data_frame()

In [ ]:
pred4_0.head()

In [ ]:
plt.plot(pred4_0.predict.values,y4eval,'o')

In [ ]:
pred4_0.predict.apply(lambda x: np.int(x+0.6)).sum(), sum(y4eval)

In [ ]:
np.vstack([pred4_0.predict.values, pred4_0.predict.values])

In [ ]:
plot_roc_curve(y4eval,np.vstack([pred4_0.predict.values, pred4_0.predict.values]).T)

In [ ]:
h2o.save_model(h2o_gbm_mdl_0,path='hdfs://nameservice1/user/kesj/ajH2O_3/p4_0/')

### Let me retry with an actual ENUM for the predictor/response variable

In [ ]:
train4_0['former'].describe()

In [ ]:
train4_0['former'] =train4_0['former'].asfactor()

In [ ]:
h2o_gbm_mdl_b0 = h2o.gbm(y = "former", x = columns_for_modeling2, training_frame = train4_0, validation_frame = test4_0, ntrees = 500 , max_depth = 4, learn_rate=0.1,distribution="bernoulli")

In [ ]:
#h2o_gbm_mdl_b0
pred4_b0 = h2o_gbm_mdl_b0.predict(eval4_frame).as_data_frame()

In [ ]:
print pred4_b0.predict.sum()
pred4_b0.head()

In [ ]:
h2o.save_model(h2o_gbm_mdl_b0,path='hdfs://nameservice1/user/kesj/ajH2O_3/p4_b0/')

In [ ]:
from sklearn import metrics
def plot_conf_matrix(y_true,y_pred,normed=True,**kwargs):
    my_c = metrics.confusion_matrix(y_true,y_pred)
    
    print metrics.matthews_corrcoef(y_true,y_pred)
    if normed:
        cm_normalized = my_c.astype('float') / my_c.sum(axis=1)[:, np.newaxis]
        my_c = cm_normalized
        plt.title('Normalized RF Classifier Confusion Matrix')
    else:
        plt.title('Random Forest Classifier Confusion Matrix')
        
    sns.heatmap(my_c, annot=True,  fmt='',cmap='Blues')
    plt.ylabel('True')
    #plt.yticks
    plt.xlabel('Assigned')
    plt.show()
    
    return

def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = metrics.roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = metrics.auc(fpr, tpr)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    return roc_auc

In [ ]:
plot_conf_matrix(y4eval,pred4_b0.predict.values)


In [ ]:
def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = metrics.roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = metrics.auc(fpr, tpr)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    return roc_auc

In [ ]:
tmp_pred = pred4_b0['p0'].copy()
tmp_pred.ix[pred4_b0[pred4_b0.predict==0].index] = pred4_b0.ix[pred4_b0[pred4_b0.predict==0].index]['p1']

In [ ]:
pred4_b0[pred4_b0.predict==0]

In [ ]:
plt.scatter(pred4_b0.p0.values,tmp_pred.values)

In [ ]:
#tmp_pred.values
plot_roc_curve(y4eval,np.vstack([1-tmp_pred.values, tmp_pred.values]).T)

In [ ]:
plot_roc_curve(y4eval, pred4_b0[['p0','p1']].as_matrix())

In [ ]:
plot_roc_curve(y4eval, pred4_b0[['p0','p1']].as_matrix())

In [ ]:
h2o_gbm_mdl_b0.varimp(return_list=True)

### Eport the POJO to a local directory space

In [ ]:
dest_dir = '/home/kesj/work/hrsepara/proc/h2o/gbm_4yr_sep_b0/'
os.chdir(dest_dir)

In [ ]:
!curl http://sfda74wbdn03.opr.statefarm.org:54321/3/h2o-genmodel.jar > h2o-genmodel.jar

In [ ]:
!curl http://sfda74wbdn03.opr.statefarm.org:54321/3/Models.java/GBM_model_python_1444141461056_19 > GBM_model_python_1444141461056_19.java


In [ ]:
#!javac -cp h2o-genmodel.jar -J-Xmx2g -J-XX:MaxPermSize=128m GBM_model_python_1444141461056_19.java

### write a function to apply this each fold in a panel

In [ ]:
rmse4,roc_auc4,gbc_mdl4 = apply_tKfold_CV2('gbc',panel4, tfold4,  cols_to_use = columns_for_modeling3, tgt_column='former',ntrees=500)#['columns_for_modeling'],em2mod['former'],)

In [ ]:
#h2o.H2ODisplay(python_obj= list(
panel4[0].ix[tfold4[0][0]].as_matrix()

## Create a set of all the current employees 
### October 28, 2015

In [ ]:
raw2002 = pd.read_csv('../eda/employees_after2001_raw.csv',dtype={'KEY':np.str})

In [ ]:
len(raw2002),len(em2)

In [ ]:
# look at the list of ACTRES1 for  ReTIREMENT
uniq_action_reasons_1 = raw2002.ACTRES1.unique()
print len(uniq_action_reasons_1)
temp_list = [str(x).split(';') for x in uniq_action_reasons_1]
from itertools import chain
act_reason_1_list = list(chain.from_iterable(temp_list))
print len(act_reason_1_list)
act_reason_1_set = set(act_reason_1_list)
print len(act_reason_1_set)
possible_retire_codes = [x for x in act_reason_1_set if ('RET' in x and  'RETURN' not in x) ]
possible_retire_codes.append('DISABILITY')
len(possible_retire_codes)
possible_retire_codes.sort()



In [ ]:
def identify_retired(x,ret_codes =possible_retire_codes):
    try:
        matched = [a for a in x.split(';') if a in ret_codes]
        if len(matched):
            return 1
        else:
            return 0
    except AttributeError:
        return 0

In [ ]:
raw2002['retired'] = raw2002.ACTRES1.apply(lambda x: identify_retired(x))

## remove the one case where it is retired but not separated

In [ ]:
print len(raw2002[~((raw2002.status == 0)& (raw2002.retired == 1))])
raw2002 = raw2002[~((raw2002.status == 0)& (raw2002.retired == 1))]
print len(raw2002), len(em2)

In [ ]:
raw2002[web_cols_to_keep].head()

In [ ]:
em2['KEY']=raw2002['KEY']


In [ ]:
web_cols_to_keep = ['KEY', 'HIRE_DT', 'BIRTHDATE', 'SAL1', 'MERIT1', 'PERF1', 'BOX1', 'SEX', 'HAVE_INS', 'HAVE_DEP']


In [ ]:
em2[['HIRE_DT','BIRTHDATE']]= raw2002[['HIRE_DT','BIRTHDATE']]#web_cols_to_keep].head()

In [ ]:
em2[web_cols_to_keep].isnull().any()

In [ ]:
all_current = raw2002[raw2002.status==0].copy()
print "there are {0} current employees".format(len(all_current))
#all_current.to_csv('current_employees.csv',index=False)

In [ ]:
all_current[web_cols_to_keep].isnull().any()

In [ ]:
all_current[['HAVE_INS','HAVE_DEP']]=all_current[['HAVE_INS','HAVE_DEP']].fillna('N').copy()
all_current[web_cols_to_keep].isnull().any()

In [ ]:
all_current[web_cols_to_keep].to_csv('../../../lib/repo/hrseparation/web_app/uploads/current.ssv',index=False,sep=';')

In [ ]:
print "For retirement:"
print "training set:\t",
print em2.retired.value_counts()/len(em2)
print "evaluation set:\t",
print em2eval.retired.value_counts()/len(em2eval)
print "For Separation"
print "training set:\t",
print em2.former.value_counts()/len(em2)
print "evaluation set:\t",
print em2eval.former.value_counts()/len(em2eval)


## CREATE a Test-train split from this dataset


In [ ]:
list_of_indices = list(range(em2002.KEY.nunique()))
print len(list_of_indices )
random.seed(823321)
#new_indices = [x for x in random.shuffle(list_of_indices)
random.shuffle(list_of_indices)#, len(list_of_indices))
em2002.index = list_of_indices # note that random.shuffle does this shuffling inplace
em2002.sort_index(inplace=True)
em2002.head()

## what are the columns included?
* examine building a model/models with minimal_input_col_list:
    - minimal_input_col_list = ['KEY','HIRE_DT','BIRTHDATE','SAL1','HAVE_INS','HAVE_DEP','EMPL_TYPE','SEX', 'MAX_RT_ANNUAL','MIN_RT_ANNUAL','PERF1','MERIT1','BOX1','INTERN','HUBIND']
* calculate Age_years & Tenure_years for each --> FeatureUnion

## October -- making this bigger
* go back to version 1.1 to see if I can construct the pipeline again


### now deal with the columns used for modeling here


In [ ]:
 = ['Age_years','Tenure_years','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP']


In [ ]:
from sklearn import cross_validation

In [ ]:
# repeat the split
# break into evaluation and build sets
print "Starting with subest of {0} employees.".format(len(em2002))
eval_fraction = 0.20
em2, em2eval = cross_validation.train_test_split(em2002,test_size=eval_fraction)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(em2eval),len(em2))

In [ ]:
em2_tgt_retired = em2.retired
em2_tgt_former = em2.former
eval_tgt_retired = em2eval.retired
eval_tgt_former = em2eval.former

### in an __ad hoc__ way convert these columns based upon our previous 'rules'

In [ ]:
other_required_cols = ['hire_tstmp','term_tstmp','birth_tstmp','former','retired']
cols_for_model_prep = []
cols_for_model_prep+=other_required_cols
cols_for_model_prep+=columns_for_modeling


In [ ]:
# replace the Y with 1 and N with 0, M with 1 and F with 0
def apply_preprocess_small(df,cols_to_use =[]):
    empl=df[cols_to_use].copy()
    #all_cols = empl.columns.tolist()
    empl['SEX'].replace({'M':1,'F':0},inplace=True)
    empl[['HAVE_INS','HAVE_DEP']]=empl[['HAVE_INS','HAVE_DEP']].replace({'Y':1,'N':0}).copy()
    empl['BOX1']=empl['BOX1'].replace({'H':3,'S':2,'L':1}).copy()
    
    # now deal with ints
    
    # fix the dollar amounts
    min_sal1 = 17621.76 #(based upon training set I have: 5 %tile cut off)
    min_min_rt_ann = 17900. # same as above
    min_max_rt_ann = 33155.70

    max_max_rt_ann = 133068.91
    min_merit1 = 0.0
    min_perf1 = 0.0
    fix_min_outlier_col_dict = {'SAL1': min_sal1, 'MERIT1': min_merit1, 'PERF1': min_perf1}
    fix_max_outlier_col_dict = {'MAX_RT_ANNUAL': max_max_rt_ann}

    # replae these values
    for key,value in fix_min_outlier_col_dict.iteritems():
        idx_to_replace = empl[empl[key]<value].index
        empl.loc[idx_to_replace,key]=value
    
    # now fill in missing with zero
    empl.fillna(0,inplace=True)
        
        
    return empl

In [ ]:
em2[cols_for_model_prep].head()

## so I want to use columns_for_modeling:
['Age_years',
 'Tenure_years',
 'SAL1',
 'MERIT1',
 'PERF1',
 'BOX1',
 'SEX',
 'HAVE_INS',
 'HAVE_DEP']
for my model building

In [ ]:
len(columns_for_modeling)

In [ ]:
kf,fp = create_temporal_kfolds(em2mod,full_date_range,5)
len(kf),len(fp)

In [ ]:
len(kf[0][0]), len(kf[0][1]), fp[0]

In [ ]:
i = 0
start_date = fp[i][0]
end_date = fp[i][1]
in_fold_idx=kf[i][0]
target_col= 'former'
cols_to_copy = ['SEX','Age_years','Tenure_years']
cols_to_copy.append(target_col)
cols_to_copy

In [ ]:
altered_fold_df = em2mod[cols_to_copy].copy()
        # adjust these
altered_fold_df.columns=['fold_mbr','adj_age','adj_tenure','adj_tgt']
altered_fold_df.fold_mbr = 0
adj_df =reset_years2(fp[i],in_fold_idx,em2mod)
altered_fold_df.head()

In [ ]:
altered_fold_df.loc[in_fold_idx,['adj_age','adj_tenure']]= adj_df.ix[in_fold_idx]
altered_fold_df.head()

In [ ]:
mypanel5 = define_target_within_x_years(em2mod,fp,kf,5,'former')

In [ ]:
mypanel5

In [ ]:
mypanel5[0].ix[kf[0][1]]

In [ ]:
fp

In [ ]:
columns_for_modeling

In [ ]:
print columns_for_modeling[2:]
X5fold = np.zeros((len(em2mod),len(columns_for_modeling),len(kf)))
y5fold = []
for i in xrange(0,len(kf)):
    X5fold[:,:-2,i]=em2mod[columns_for_modeling[2:]].as_matrix().astype(np.float)
    X5fold[:,-2:,i]=mypanel5[i][['adj_age','adj_tenure']]
    my_y=mypanel5[i][['adj_tgt']].as_matrix().astype(np.int)
    y5fold.append(my_y)

In [ ]:
evalmod = apply_preprocess_small(em2eval,cols_for_model_prep)

In [ ]:
## now apply each model to my eval set
def evaluate_models(model_list,Xeval):
    eval_pred_class = np.zeros((len(Xeval),len(model_list)))
    eval_pred_proba = np.zeros((len(Xeval),2,len(model_list)))

    for i,mdl in enumerate(model_list):
        eval_proba = mdl.predict_proba(Xeval)
        eval_pred_class[:,i]=mdl.predict(Xeval)
        eval_pred_proba[:,:,i]=eval_proba
    #print np.shape(eval_prediction_proba3)
    return eval_pred_class, eval_pred_proba

## Reworking this
1. preprocess the test data 
    0. transform all to numbers and fill in missing (```apply_preprocess_small```)
    1. define the time_range
    2. define the time fields (both timestamps and temporally dependent columns)
        1. default_tstmps
        2. _years columns: Age_years & Tenure_years
    3. define the columns to use (for modeling)
2. setup_tfold_models
    * create_temporal_kfolds
    * define_target_within_x_years
    * 
    

In [ ]:
len(kf)

In [ ]:
contrary_case = []
contrary_case += default_tstmps 
contrary_case.append('jobchg_tstmp')


In [ ]:
if [a for a in default_tstmps if a in em2mod.columns] == default_tstmps

In [ ]:
if not [a for a in contrary_case if a in em2mod.columns] == contrary_case:
    print "some columns in contrary_case are missing."

In [ ]:
pan1 = transform_df_to_tfold(em2mod,kf,fp,5,columns_for_modeling,'former',default_tstmps,['Age_years','Tenure_years'])
                      
    

In [ ]:
pan1.items


In [ ]:
[sum(pan1[a].former.isnull()) for a in xrange(0,len(pan1.items.tolist()))]

In [ ]:
pan1[8].head()

In [ ]:
pan1[0].head()

In [ ]:
em2mod.head()

In [ ]:
sum(pan1[0].former.isnull()), len(kf[0][1]), len(kf[0][0])

## 10/01/15
try gradient boosted classifier

In [ ]:
rmse5,roc_auc5,rfmdl5 = apply_tKfold_CV2('gbc',pan1, kf,  cols_to_use = columns_for_modeling, tgt_column='former')#['columns_for_modeling'],em2mod['former'],)

## Stopped here for LABOR DAY



In [ ]:
panel4, tfold4 = setup_tfolds(em2mod, 4, columns_for_modeling, tgt_value = 'former',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
   

In [ ]:
for training, testing in tfold4: #len(tfold4[0][1]), len(tfold4[0][0])
    print len(training),len(testing)

# build a method/function to apply a model to the temporal_kfold

In [ ]:
#metrics.mean_squared_error
from sklearn import ensemble
from sklearn import tree

In [ ]:
def apply_tKfold_CV(model, in_panel, tkfolds, cols_to_use, tgt_column = 'former' ):
    """ Calculate the rmse for each cross-validated temporalFold
    
    Parameters:
        model - a scikit learn model
        in_panel  - a pandas Panel with the observed input data adjusted 
        in_y - a pandas Series with the observed outcome
        k - number of cross validation folds (test set will be 1/k of the data)
    """
    RMSE = []
    for fold_id, indices in enumerate(tkfolds):
        training = indices[0]
        testing = indices[1]
        print fold_id, len(training),len(testing)
        X_train, X_test = in_panel[fold_id][cols_to_use].ix[training].as_matrix(), in_panel[fold_id][cols_to_use].ix[testing].as_matrix()
        y_train, y_test = in_panel[fold_id][tgt_column].ix[training].as_matrix(), in_panel[fold_id][tgt_column].ix[testing].as_matrix()
        
        # Train the model
        model.fit(X_train,y_train)
        # use the model to predict output
        y_fitted = model.predict(X_test)
        RMSE.append(np.sqrt(metrics.mean_squared_error(y_test,y_fitted)))
    # leave the model fit to the entire dataset
    #model.fit(in_x,in_y)
    
    return RMSE
        

In [ ]:
def apply_tKfold_CV2(modeltype, in_panel, tkfolds, cols_to_use, tgt_column = 'former', ntrees=100):
    """ Calculate the rmse for each cross-validated temporalFold
    
    Parameters:
        model - a scikit learn model name
        in_panel  - a pandas Panel with the observed input data adjusted 
        tgt_column -- the target variable
        cols_to_use -- the input variables to be used in modeling
        tkfolds -- teh temporal timefolds (list of dimension 2)
    outputs:
        models --> the list of fit models
        RMSE --> the rmse error
        
    """
    RMSE = []
    roc_auc = []
    models = []
    
    
    
    for fold_id, indices in enumerate(tkfolds):
        training = indices[0]
        testing = indices[1]
        #print fold_id, len(training),len(testing)
        X_train, X_test = in_panel[fold_id][cols_to_use].ix[training].as_matrix(), in_panel[fold_id][cols_to_use].ix[testing].as_matrix()
        y_train, y_test = in_panel[fold_id][tgt_column].ix[training].as_matrix(), in_panel[fold_id][tgt_column].ix[testing].as_matrix()
        if modeltype == 'rfc':
            model = ensemble.RandomForestClassifier(n_estimators=ntrees, max_features='auto', oob_score=True)
        elif modeltype=='bgc':
            model=ensemble.GradientBoostingClassifier(n_estimators=ntrees,max_features='auto')
        else:
            model=tree.DecisionTreeClassifier()
            
        # Train the model
        model.fit(X_train,y_train)
        # use the model to predict output
        y_fitted = model.predict(X_test)
        RMSE.append(np.sqrt(metrics.mean_squared_error(y_test,y_fitted)))
        roc_auc.append(metrics.roc_auc_score(y_test,y_fitted))
        models.append(model)
    # leave the model fit to the entire dataset
    #model.fit(in_x,in_y)
    
    return RMSE,roc_auc,models
        

### Now setup protocol for generating the models

In [ ]:
#random_forest = ensemble.RandomForestClassifier(n_estimators=100, max_features='auto')

In [ ]:
rmse4,roc_auc4,rfmdl4 = apply_tKfold_CV2('rfc',panel4, tfold4, cols_to_use = columns_for_modeling, tgt_column='former')#['columns_for_modeling'],em2mod['former'],)

In [ ]:
rmse4


In [ ]:
roc_auc4

In [ ]:
for a in xrange(0,len(panel4)):
    print a, panel4[a][panel4[a].former==1].former.sum()

## apply to the hold-out (EVAL) set

In [ ]:
pd.DataFrame(roc_auc4).boxplot()

In [ ]:
[(a,rfmdl4[a].oob_score_) for a in xrange(0,len(rfmdl4))]

In [ ]:
feature_list= pan1[0].columns
feature_list = columns_for_modeling
feature_list
fi_5df = create_fi_df(rfmdl5,columns_for_modeling)

In [ ]:
fi_4df.head()

In [ ]:
fval_cols = [a for a in fi_4df.columns if 'value' in a]
fi_4df[fval_cols].T.boxplot()
plt.title('Feature Importances for 4year former')
plt.ylabel('Feature Importances in Tfolds')

In [ ]:
X4eval, y4eval = adjust_eval_by_x_years(evalmod,4,columns_for_modeling)

In [ ]:
X4evalb, y4evalb = adjust_eval_by_x_years(evalmod,4,columns_for_modeling)

In [ ]:
beval_pred4class, beval_pred4proba = evaluate_models(rfmdl4,X4evalb)

In [ ]:
#plot_roc_curve(y4eval,map(eval_pred4class.mean(),np.int))
plot_roc_curve(y4evalb,beval_pred4proba[:,:,:].mean(axis=2))

In [ ]:
beval_pred4class.mean(axis=1)

In [ ]:
beval_pred4class.min(axis=1)

In [ ]:
plot_conf_matrix(y4evalb,map(np.int,beval_pred4class.mean(axis=1)+0.7))

In [ ]:
#plot_roc_curve(y4eval,map(eval_pred4class.mean(),np.int))
plot_roc_curve(y4eval,eval_pred4proba[:,:,:].mean(axis=2))

In [ ]:
plot_conf_matrix(y4eval,map(np.int,eval_pred4class.mean(axis=1)))

In [ ]:
evalmod.head()

In [ ]:
len(y5eval),len(evalmod)

In [ ]:
len(evalmod[evalmod.Tenure_years <=5.0+.5])

In [ ]:
panel5[0].head()

In [ ]:
X5eval

In [ ]:
evalmod.former.sum()

In [ ]:
print y5eval.sum()
y5eval

In [ ]:
X5eval, y5eval = adjust_eval_by_x_years(evalmod,5,columns_for_modeling)

In [ ]:
## try for 5year
panel5, tfold5 = setup_tfolds(em2mod, 5, columns_for_modeling, tgt_value = 'former',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
rmse5,roc_auc5,rfmdl5 = apply_tKfold_CV2('gbc',panel5, tfold5, cols_to_use = columns_for_modeling, tgt_column='former')#['columns_for_modeling'],em2mod['former'],)
X5eval, y5eval = adjust_eval_by_x_years(evalmod,5,columns_for_modeling)

In [ ]:
## Repeat for more trees
rmse5b,roc_auc5b,rfmdl5b = apply_tKfold_CV2('gbc',panel5, tfold5, cols_to_use = columns_for_modeling, tgt_column='former',ntrees=500)#['columns_for_modeling'],em2mod['former'],)
X5evalb,y5evalb = adjust_eval_by_x_years(evalmod,5,columns_for_modeling)

In [ ]:
cv_scores5b = pd.DataFrame()
cv_scores5b['rmse'] = rmse5b#pd.DataFrame(rmse5)#.boxplot()
cv_scores5b['roc_auc']=roc_auc5b
#pd.DataFrame(roc_auc5).boxplot()
cv_scores5b.boxplot()

In [ ]:
cv_scores5 = pd.DataFrame()
cv_scores5['rmse'] = rmse5#pd.DataFrame(rmse5)#.boxplot()
cv_scores5['roc_auc']=roc_auc5
#pd.DataFrame(roc_auc5).boxplot()
cv_scores5.boxplot()

In [ ]:
rmse5, rmse5b

In [ ]:
ev5df = pd.DataFrame()
ev5df['avg'] = eval_pred5class.mean(axis=1)
#ev5df.head()
ev5df['min'] = eval_pred5class.min(axis=1)
ev5df['max']= eval_pred5

In [ ]:
eval_pred5class, eval_pred5proba = evaluate_models(rfmdl5,X5eval)
plot_conf_matrix(y5eval,map(np.int,eval_pred5class.mean(axis=1)))

In [ ]:
plot_roc_curve(y5eval,eval_pred5proba[:,:,:].mean(axis=2))

In [ ]:
eval_pred5classb, eval_pred5probab = evaluate_models(rfmdl5b,X5eval)
plot_conf_matrix(y5eval,map(np.int,eval_pred5classb.mean(axis=1)))

In [ ]:
np.sum(map(np.int,eval_pred5classb.mean(axis=1))), len(eval_pred5classb)

In [ ]:
plot_roc_curve(y5eval,eval_pred5probab[:,:,:].mean(axis=2))
plot_roc_curve(y5eval,eval_pred5proba[:,:,:].mean(axis=2))

In [ ]:
fi_5df = create_fi_df(rfmdl5,columns_for_modeling)
val_cols = [a for a in fi_5df.columns if 'value' in a]
fi_5df[val_cols].T.boxplot()
plt.title('Feature Importances for 5year former')
plt.ylabel('Feature Importances in Tfolds')

In [ ]:
!mkdir 'gbc_5sep_pkl'
%cd '/home/kesj/work/hrsepara/eda/gbc_5sep_pkl/'

In [ ]:
import pickle
pickle.dump(rfmdl5,open('gbc5.pkl','wb'))
#len(flist_5)

## Try loading the previously pickled models

In [ ]:
%cd '/home/kesj/work/hrsepara/eda/jl_5yr_100auto/'

In [ ]:
stored_mdl = jl.load('frf100.pkl')

In [ ]:
Seval_pred5class,Seval_pred5proba = evaluate_models(stored_mdl,X5eval)

In [ ]:
plot_roc_curve(y5eval,Seval_pred5proba[:,:,:].mean(axis=2))

## repeat for 3 years:

In [ ]:
panel3, tfold3 = setup_tfolds(em2mod, 3, columns_for_modeling, tgt_value = 'former',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
rmse3,roc_auc3,rfmdl3 = apply_tKfold_CV2('gbc',panel3, tfold3, cols_to_use = columns_for_modeling, tgt_column='former')#['columns_for_modeling'],em2mod['former'],)
X3eval, y3eval = adjust_eval_by_x_years(evalmod,3,columns_for_modeling)

In [ ]:
eval_pred3class, eval_pred3proba = evaluate_models(rfmdl3,X3eval)
plot_conf_matrix(y3eval,map(np.int,eval_pred3class.mean(axis=1)))

In [ ]:
plot_roc_curve(y3eval,eval_pred3proba[:,:,:].mean(axis=2))

In [ ]:
np.shape(eval_pred3proba)

In [ ]:
for i in xrange(0,len(panel3)):
    #fpr,tpr,treshholds = roc_
    plot_roc_curve(y3eval,eval_pred3proba[:,:,i])
plot_roc_curve(y3eval,eval_pred3proba[:,:,:].mean(axis=2))

In [ ]:
plot_roc_curve(y3eval,eval_pred3proba[:,:,0])

In [ ]:
panel2, tfold2 = setup_tfolds(em2mod, 2, columns_for_modeling, tgt_value = 'former',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
rmse2,roc_auc2,rfmdl2 = apply_tKfold_CV2('rfc',panel2, tfold2, cols_to_use = columns_for_modeling, tgt_column='former')#['columns_for_modeling'],em2mod['former'],)
X2eval, y2eval = adjust_eval_by_x_years(evalmod,2,columns_for_modeling)

In [ ]:
eval_pred2class, eval_pred2proba = evaluate_models(rfmdl2,X2eval)
plot_conf_matrix(y2eval,map(np.int,eval_pred2class.mean(axis=1)))

In [ ]:
panel1, tfold1 = setup_tfolds(em2mod, 1, columns_for_modeling, tgt_value = 'former',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
rmse1,roc_auc1,rfmdl1 = apply_tKfold_CV2('rfc',panel1, tfold1, cols_to_use = columns_for_modeling, tgt_column='former')#['columns_for_modeling'],em2mod['former'],)
X1eval, y1eval = adjust_eval_by_x_years(evalmod,1,columns_for_modeling)

In [ ]:
eval_pred1class, eval_pred1proba = evaluate_models(rfmdl1,X1eval)
plot_conf_matrix(y1eval,map(np.int,eval_pred1class.mean(axis=1)))

In [ ]:
for i in xrange(0,len(panel1)):
    #fpr,tpr,treshholds = roc_
    plot_roc_curve(y1eval,eval_pred1proba[:,:,i])
plot_roc_curve(y1eval,eval_pred1proba[:,:,:].mean(axis=2))

In [ ]:
em2mod.columns

In [ ]:
Rpanel2[0].head()

## Try for 'retired'

In [ ]:
today = pd.to_datetime('2015-01-01')
today

In [ ]:

evalmod[(evalmod['retired']==1) & (today - evalmod['term_tstmp']<= 1.5)]

In [ ]:
def adjust_eval_by_x_yearsR(df,year_val,modeling_columns,endtime,tstmp_cols=default_tstmps,target_col='retired'):
    # construct
  
    ## set up method to assess the eval set
    print "There are {0} elements in the evaluation set".format(len(df))
   
    print "original target variable value counts:", df[target_col].value_counts()
    # restructure to deal with time_frame retirement (target variable)
    yr_cut_val = year_val+0.5
    df['rel_yrs'] = df['term_tstmp'].apply(lambda x: (endtime - x).days/365.25)
    # index of those that actually accomplish target within timeframe (allow 0.5 additional years)
    #eval_within_time_target_index = df[(df[target_col]==1) & (endtime - df['term_tstmp'] <= yr_cut_val)].index
    eval_within_time_target_index = df[(df[target_col]==1) & (df['rel_yrs'] <= yr_cut_val)].index
    # exclude indices that are active and have tenure less than this time
    eval_excluded_index = df[(df[target_col]==0) & (df.Tenure_years  <= yr_cut_val)].index
    
    # the rest become my not-terminated set
    eval_active_index = set(df.index) - set(eval_within_time_target_index) - set(eval_excluded_index)
    print len(eval_excluded_index),len(eval_within_time_target_index), len(eval_active_index)
    eval_idx_to_use =df.ix[set(df.index)-set(eval_excluded_index)].index
    #len(eval_idx_to_use)
    # reset the target to 0 for active
    eval_new_target = df[target_col].copy()
    eval_new_target.ix[eval_active_index] = 0
    print "new target variable value counts: "
    print eval_new_target.ix[eval_idx_to_use].value_counts()
    print "_____"
    y_eval = eval_new_target.ix[eval_idx_to_use].as_matrix().astype(np.int) # true values
    eval_adj_tenure = df.ix[eval_idx_to_use].Tenure_years.apply(lambda x: x-year_val if (x>float(year_val)) else 0).values
    print len(eval_adj_tenure), len(y_eval)
    # now adjust age by length of time; use hire_age if not in set to use.
    eval_adj_age = df.ix[eval_idx_to_use].Age_years.apply(lambda x: x-year_val)
    hire_age = (df.ix[eval_within_time_target_index]['birth_tstmp']-df.ix[eval_within_time_target_index]['hire_tstmp'])/np.timedelta64(1,'Y')#.days/days_in_year
    eval_adj_age.ix[eval_within_time_target_index] = hire_age #df.ix[eval_within_time_target_index]['birth_tstmp'#df_dates['hire_age']
    
    # construct the evaluation X matrix
    print "input matrix has {0} features".format(len(modeling_columns))
    Xeval = np.zeros((len(eval_idx_to_use),len(modeling_columns)))
    # drop 'Age_years' and Tenure_years from the list
    cols_to_use = []
    cols_to_use+=modeling_columns
    cols_to_use.remove('Age_years')
    cols_to_use.remove('Tenure_years')#.copy()
    """
    Xeval[:,:-2] = df.ix[eval_idx_to_use][cols_to_use].as_matrix().astype(np.float)
    # now put the adjusted tenure and ages into this matrix
    Xeval[:,-2] = eval_adj_age.values
    Xeval[:,-1]=eval_adj_tenure
    #print len(modeling_columns),np.shape(Xeval)
    """
    # this version matches the changed ording of columns
    Xeval[:,0]=eval_adj_age.values
    Xeval[:,1]=eval_adj_tenure
    Xeval[:,2:] = df.ix[eval_idx_to_use][cols_to_use].as_matrix().astype(np.float)
    return Xeval, y_eval

In [ ]:
Rpanel2, Rtfold2 = setup_tfolds(em2mod, 2, columns_for_modeling, tgt_value = 'retired',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
Rrmse2,Rroc_auc2,Rrfmdl2 = apply_tKfold_CV2('rfc',Rpanel2, Rtfold2, cols_to_use = columns_for_modeling, tgt_column='retired')#['columns_for_modeling'],em2mod['former'],)


In [ ]:
Reval_pred2class,Reval_pred2proba = evaluate_models(Rrfmdl2,RX2eval)
plot_roc_curve(Ry2eval,Reval_pred2proba[:,:,:].mean(axis=2))

In [ ]:
Rpanel5, Rtfold5 = setup_tfolds(em2mod, 5, columns_for_modeling, tgt_value = 'retired',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
Rrmse5,Rroc_auc5,Rrfmdl5 = apply_tKfold_CV2('rfc',Rpanel5, Rtfold5, cols_to_use = columns_for_modeling, tgt_column='retired')#['columns_for_modeling'],em2mod['former'],)
RX5eval, Ry5eval = adjust_eval_by_x_yearsR(evalmod,5,columns_for_modeling, today,target_col = 'retired')

In [ ]:
Reval_pred5class,Reval_pred5proba = evaluate_models(Rrfmdl5,RX5eval)
plot_roc_curve(Ry5eval,Reval_pred5proba[:,:,:].mean(axis=2))

In [ ]:
Rpanel4, Rtfold4 = setup_tfolds(em2mod,4, columns_for_modeling, tgt_value = 'retired',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
Rrmse4,Rroc_auc4,Rrfmdl4 = apply_tKfold_CV2('rfc',Rpanel4, Rtfold5, cols_to_use = columns_for_modeling, tgt_column='retired')#['columns_for_modeling'],em2mod['former'],)


In [ ]:
RX4eval, Ry4eval = adjust_eval_by_x_yearsR(evalmod,4,columns_for_modeling, today,target_col = 'retired')
Reval_pred4class,Reval_pred4proba = evaluate_models(Rrfmdl4,RX4eval)
plot_roc_curve(Ry4eval,Reval_pred4proba[:,:,:].mean(axis=2))

In [ ]:
Rpanel3, Rtfold3 = setup_tfolds(em2mod, 3, columns_for_modeling, tgt_value = 'retired',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
Rrmse3,Rroc_auc3,Rrfmdl3 = apply_tKfold_CV2('rfc',Rpanel3, Rtfold3, cols_to_use = columns_for_modeling, tgt_column='retired')#['columns_for_modeling'],em2mod['former'],)


In [ ]:
RX3eval, Ry3eval = adjust_eval_by_x_yearsR(evalmod,3,columns_for_modeling,today, target_col = 'retired')
Reval_pred3class,Reval_pred3proba = evaluate_models(Rrfmdl3,RX3eval)
plot_roc_curve(Ry3eval,Reval_pred3proba[:,:,:].mean(axis=2))

In [ ]:
Rpanel1, Rtfold1 = setup_tfolds(em2mod, 1, columns_for_modeling, tgt_value = 'retired',
                 time_cols=['Age_years','Tenure_years'], tstmp_cols = default_tstmps, date_range=full_date_range)
Rrmse1,Rroc_auc1,Rrfmdl1 = apply_tKfold_CV2('rfc',Rpanel5, Rtfold5, cols_to_use = columns_for_modeling, tgt_column='retired')#['columns_for_modeling'],em2mod['former'],)


In [ ]:
evalmod.head()

In [ ]:
RX1eval[:10]

In [ ]:
len(evalmod[(evalmod['retired']==1) & (today - evalmod['term_tstmp'] <= 5.5*365.25)])

In [ ]:
5.5*365.25

In [ ]:
sum((today - evalmod['term_tstmp']) <= 5.5*365.25)

In [ ]:
RX1eval, Ry1eval = adjust_eval_by_x_yearsR(evalmod,1,columns_for_modeling, today, target_col = 'retired')
Reval_pred1class,Reval_pred1proba = evaluate_models(Rrfmdl1,RX1eval)
plot_roc_curve(Ry1eval,Reval_pred1proba[:,:,:].mean(axis=2))

## Save the models

In [ ]:
%cd ../

In [ ]:
!mkdir save_models_1

In [ ]:
%cd 'save_models_1/'

In [ ]:
!mkdir sep
!mkdir ret

In [ ]:
## define the base directories
case_dir = ['sep/','ret/']
## define the model cases
nyrs = map(str,np.arange(1,6))
base='rfmdl'
model_listing = []
pkl_file_dict = {}
#dump_file_listing= []
for case in case_dir:
    #!cd {case_dir}
    for yr in nyrs:
        #dir_name = case+yr+'/'
        #!mkdir {yr}
        #!cd {yr}
        model_name = base+yr
        file_name = 'rf'
        if case == 'ret/':
            model_name = 'R'+model_name
            file_name = 'R'+file_name
        file_name+=yr
        file_name+='.pkl'
        
        print dir_name, model_name, file_name
        model_listing.append(model_name)
        pkl_file_dict[model_name]=file_name
        #file_list = jl.dump(model_name,file_name)
        #dump_file_listing.append(file_list)
        #!cd ../
    #!cd ../

In [ ]:
store_dir = '/home/kesj/work/hrsepara/save_models_1/'
dump_file_listing = []
for mdl in model_listing:
    if mdl.startswith('R'):
        base_dir = store_dir + 'ret'
        #!cd {base_dir}#/home/kesj/work/hrsepara/save_models_1/ret        
    else:
        base_dir = store_dir + 'sep'
    
    os.chdir(base_dir)#!cd {base_dir}
    yr = mdl[-1]
    #os.makedirs(yr)#!mkdir {yr}
    #os.chdir(yr)#!cd {yr}
    print yr,mdl
    #file_list =jl.dump(mdl,pkl_file_dict[mdl])
    #dump_file_listing.append(file_list)

In [ ]:
os.chdir('/home/kesj/work/hrsepara/save_models_1/sep')

In [ ]:
os.mkdir('1')
os.chdir('1')

In [ ]:
dump_file_listing = []

In [ ]:
flist = jl.dump(rfmdl1,'rf1.pkl')
dump_file_listing.append(flist)

In [ ]:
os.chdir('/home/kesj/work/hrsepara/save_models_1/sep')
os.mkdir('2')
os.chdir('2')
flist = jl.dump(rfmdl2,'rf2.pkl')
dump_file_listing.append(flist)

In [ ]:
os.chdir('/home/kesj/work/hrsepara/save_models_1/sep')
os.mkdir('3')
os.chdir('3')
flist = jl.dump(rfmdl3,'rf3.pkl')
dump_file_listing.append(flist)

In [ ]:
os.chdir('/home/kesj/work/hrsepara/save_models_1/sep')
os.mkdir('4')
os.chdir('4')
flist = jl.dump(rfmdl4,'rf4.pkl')
dump_file_listing.append(flist)


In [ ]:
os.chdir('/home/kesj/work/hrsepara/save_models_1/sep')
os.mkdir('5')
os.chdir('5')
flist = jl.dump(rfmdl5,'rf5.pkl')
dump_file_listing.append(flist)

In [ ]:
k=5
for yr in nyrs:
    os.chdir('/home/kesj/work/hrsepara/save_models_1/ret')
    os.mkdir(yr)
    os.chdir(yr)
    flist = jl.dump(,'rf5.pkl')
dump_file_listing.append(flist)

In [ ]:
os.chdir('/home/kesj/work/hrsepara/save_models_1/ret')

In [ ]:
os.mkdir('1')
os.chdir('1')
flist = jl.dump(Rrfmdl1,'Rrf1.pkl')
dump_file_listing.append(flist)

os.chdir('/home/kesj/work/hrsepara/save_models_1/ret')
os.mkdir('2')
os.chdir('2')
flist = jl.dump(Rrfmdl2,'Rrf2.pkl')
dump_file_listing.append(flist)

In [ ]:
os.chdir('/home/kesj/work/hrsepara/save_models_1/ret')
os.mkdir('3')
os.chdir('3')
flist = jl.dump(Rrfmdl3,'Rrf3.pkl')
dump_file_listing.append(flist)

os.chdir('/home/kesj/work/hrsepara/save_models_1/ret')
os.mkdir('4')
os.chdir('4')
flist = jl.dump(Rrfmdl4,'Rrf4.pkl')
dump_file_listing.append(flist)

os.chdir('/home/kesj/work/hrsepara/save_models_1/ret')
os.mkdir('5')
os.chdir('5')
flist = jl.dump(Rrfmdl5,'Rrf5.pkl')
dump_file_listing.append(flist)

In [ ]:
panel4[0]

### Aside:
* KFold within sklearn does the following 

In [ ]:
std_folds = cross_validation.KFold(n=len(em2mod),n_folds=3)

In [ ]:
for std_tr,std_test in std_folds:
    print len(std_tr), len(std_test)

## now build t_fold models


In [ ]:
from sklearn import ensemble

In [ ]:
len(tfold5)

In [ ]:
%%time
five_rf_mdl_100A = []
for i in xrange(0,len(kf)):
    train_y = y5fold[i].flatten()[kf[i][0]]
    train_X = X5fold[kf[i][0],:,i]
    rfmdl = ensemble.RandomForestClassifier(n_estimators=100,max_features='auto',n_jobs=1)
    rfmdl.fit(train_X,train_y)
#baseline_singleRFC = ensemble.RandomForestClassifier(n_jobs=50,n_estimators=500,max_features=None)
#baseline_singleRFC.fit(X,y_term)
#baseline_singleRFC_importances= baseline_singleRFC.feature_importances_
    five_rf_mdl_100A.append(rfmdl)

In [ ]:
five_rf_mdl_100A[0].feature_importances_

In [ ]:
X5eval, y5eval = adjust_eval_by_x_years(evalmod,5,columns_for_modeling)

In [ ]:
eval_pred5class,eval_pred5proba = evaluate_models(five_rf_mdl_100A,X5eval)

In [ ]:
plot_conf_matrix(y5eval,map(np.int,eval_pred5class.mean(axis=1)))

In [ ]:
#plot_roc_curve(y3eval,eval_prediction_proba3[:,])
plot_roc_curve(y5eval,eval_pred5proba[:,:,:].mean(axis=2))
plt.ylim([0,1.01])

### function to help explore the features

In [ ]:
# function to push feature_importances for a set of RF models into a dataframe
def create_fi_df(mdl_list,feature_names):
    list_feature_importances = []
    col_list = []
    for i,mdl in enumerate(mdl_list):
        list_feature_importances.append(plotFI(mdl,feature_names,show_plot=False))
        col_list.append('fold'+str(i)+'_value')
        col_list.append('fold'+str(i)+'_std')

    fi_df = pd.concat(list_feature_importances,axis=1)
    # create column headings
    fi_df.columns = col_list
    # create the average of the values
    value_cols = [x for x in col_list if x.endswith('value')]
    
    fi_df['avg_val']=fi_df[value_cols].mean(axis=1)
    fi_df['avg_variance']=fi_df[value_cols].std(axis=1)
#t2_eval_fi_df[['avg_val','avg_std']].sort('avg_val',ascending=False)
    return fi_df

def plotFI(forest,featureNames=[],show_plot=True):#,autoscale=True,headroom=0.05):
    """
    forest is the model to be graphed.
    featureNames is the list of features to be displayed
    
    """
    #if autoscale:
    #    x_scale = forest.feature_importances_.max()+ headroom
    #else:
    #    x_scale = 1
    
    featureImportances=forest.feature_importances_
    # sort the importances from biggest to least
    indices = np.argsort(featureImportances)[::-1]
    estimators = forest.estimators_
    # calculate the variance over the forest 
    
    std = np.std([tree.feature_importances_ for tree in estimators],axis=0)
    # print summary statement
    nfeatures = len(featureImportances)
    print("Number of Features: %d" % (nfeatures))
    print("Number of Trees: %d" %(len(estimators)))
    
    #print featureNames
    if len(featureNames)==0:
        featureNames = map(str,indices)
    
    fN2 = [featureNames[a] for a in indices]
    print("Feature ranking:")

    for f in range(len(indices)):
        print("%d. feature %d=%s (%f)" % (f + 1, indices[f], featureNames[indices[f]],featureImportances[indices[f]]))

    # Plot the feature importances of the forest
    # define a cutoff in terms of feature_importance
    if nfeatures <= 30:
        kfeatures = nfeatures # keep all if smaller than 30
    else:
        kfeatures = 30
        
    kindices = indices[:kfeatures]
    if show_plot:
        plt.title("Feature importances")
        plt.barh(range(len(kindices)), featureImportances[kindices],
           color="steelblue", xerr=std[kindices], align="center",ecolor='k')#,lw=2)
    
        plt.yticks(range(len(kindices)),fN2)
        #grid(True)
    
    c1 = 'value'
    c2 = 'std'
    tdata = np.vstack([featureImportances[indices],std[indices]])
    df = pd.DataFrame(data = tdata.T,index=fN2,columns=[c1,c2])
    return df

In [ ]:
feature_list = [a for a in columns_for_modeling if 'years' not in a]
feature_list += columns_for_modeling[:2]
feature_list

In [ ]:
#[e for e in tree0.estimators_]
np.shape(tree0.indices_)



In [ ]:
fi_5df = create_fi_df(five_rf_mdl_100A,feature_list)
#tree0 = rfmdl[0]

In [ ]:
fi_5df.plot(kind='bar',y='avg_val')#,std='avg_var')

In [ ]:
fi_5df.plot(kind='bar',y='avg_variance')

##save this model to disk

In [ ]:
%cd '/home/kesj/work/hrsepara/eda/jl_5yr_100auto/'

In [ ]:
import joblib as jl

In [ ]:
%cd ../
!du -h 'jl_5yr_100auto'

## work through evaluating based upon a set of existing models
* I want to save all the separations to a data frame and then save it to disk
* likewise with the retirements

In [ ]:
#os.chdir('../../')

os.chdir(stgdir1local)
%ls

In [ ]:
MODEL_BASE_PATH='../save_models_1/'
pred_cases = ['sep/','ret/']
pred_years = map(np.str,np.arange(1,6))


In [ ]:
sep_proba_df = []
ret_proba_df = []
import joblib as jl


In [ ]:
def evaluate_models_simple(model_list,X,mode='mean',offset=0):
    """ Function to apply a set of models to a given input and generate the predicted value(s)
    :param model_list --> input list of models
    :param X --> input array to apply models to
    :param mode --> what sort of output to return; default is mean
    intermediates
        eval_pred_class --> array of classification prediction for each model
        eval_pred_proba --> array of predicted probabilities for each model
    """
    import numpy as np
    eval_pred_class = np.zeros((len(X),len(model_list)))
    eval_pred_proba = np.zeros((len(X),2,len(model_list)))

    for i,mdl in enumerate(model_list):
        eval_proba = mdl.predict_proba(X)
        eval_pred_class[:,i]=mdl.predict(X)
        eval_pred_proba[:,:,i]=eval_proba
    if mode == 'mean':
        # average the probabilities (for class=1) and return the mean predicted probability
        prediction = np.mean(eval_pred_proba[:,1,:],axis=1)
        #eval_pred_proba[:,1,:].mean(axis=2)

    elif mode == 'class': # return the desired class prediction
        prediction = map(np.int,eval_pred_class.mean(axis=1))
    return prediction

In [ ]:
#del X
np.shape(X5eval)
X = X5eval[30:80,:]
np.shape(X)

In [ ]:
sep_proba_df = []
for idx,pyr in enumerate(pred_years):
    pkl_name ='rf'+pyr+'.pkl'#5.pkl'
    path_name =MODEL_BASE_PATH+pred_cases[0]+pyr+'/'
    print path_name, pkl_name
    #path_name = MODEL_BASE_PATH+pred_cases[0]+"5/"#pred_years[0]+'/'
        # load a model, evaluate it and return the 'average' probability for each person
        #abs_mdl_name = os.path.abspath(path_name+pkl_name)
    mdl_name = path_name+pkl_name
    stored_mdl = jl.load(mdl_name)
    stored_prediction = evaluate_models_simple(stored_mdl,X)#stored_pred,stored_pred_proba = evaluate_models(stored_mdl,X)

    df = pd.DataFrame(stored_prediction.T)
    df.columns=['sep'+pyr+'yr']
    #df.#print stored_prediction
    sep_proba_df.append(df)



In [ ]:
len(sep_proba_df)
#sep_df.shape()

In [ ]:
sep_df = pd.concat([a for a in sep_proba_df],axis=1)
sep_df.head()

In [ ]:
#sep_df.T.plot()

In [ ]:
#df.rename(columns=['sep'+pyr+'yr'],inplace=True)
df.columns=['sep'+pyr+'yr']
df.head()

In [ ]:
#sep_proba_df = sep_proba_df[1:]
pd.concat([a for a in sep_proba_df],axis=1)

## Some miscellaneous, experimental visualizations of the predictions

In [ ]:
print len(np.mean(eval_pred5proba[:,1,:],axis=1))
ev5df = pd.DataFrame(eval_pred5proba[:,1,:])
ev5df.head()
#ev5df['avg'] = eval_pred5proba[:,0,:].mean(axis=2)
#ev5df['avg'] = eval_pred5proba[:,0,:].min(axis=2)

In [ ]:
nshow = 21
ev5df.head(nshow).T.boxplot(vert=False)
plt.xlabel('predicted probability of separation')
plt.ylabel('employee')
plt.title('5 year window')
#plt.plot(x=y5eval[:30],y=np.arange(0,30),color='darkgoldenrod',marker='*')
for x,y in np.vstack([y5eval[:nshow],np.arange(0,nshow)]).T:
    plt.plot(x,y,'D',color='darkgoldenrod')
for x,yhat in np.vstack([map(np.int,eval_pred5class.mean(axis=1))[:nshow],np.arange(0,nshow)]).T:
    plt.plot(x,yhat,'o',color='darkorchid')

In [ ]:
p_class = ev5df.head(nshow).mean(axis=1) #map(np.int,ev5df.head(nshow).mean(axis=1))


In [ ]:
p_class = p_class.apply(lambda x: round(x,0))
#p_class['color']= 'red'
p_class

In [ ]:
cname = ['red']*21


In [ ]:


sns.violinplot(ev5df.head(nshow).T,vert=False)
plt.xlabel('Probability of separation')
plt.ylabel('employee')
plt.xlim([0,1])

## Create a scoring measure for different models

In [ ]:
def perform_kfold_cross_validation(model, all_X, all_y, k=5):
    """Calculate root mean squared error for each cross-validation fold.
    
    Parameters:
        model - a scikit learn model
        all_X - a pandas DataFrame with the observed input data
        all_y - a pandas Series with the observed outcome
        k - number of cross validation folds (test set will be 1/k of the data)
    
    Return value:
        An array of length 'k' with the root mean squared error
        for each fold.
    """
    # 'folds' is a generator that will yield pairs of arrays (train, test)
    # selecting row numbers for training/testing
    folds = cross_validation.KFold(n=len(all_y), n_folds=k)
    RMSE = []    # root mean squared errors
    # Loop over the cross-validation folds
    for training, testing in folds:
        # Get the training and test splits
        training = all_X.index[training]
        testing = all_X.index[testing]
        X_train, X_test = all_X.ix[training], all_X.ix[testing]
        y_train, y_test = all_y.ix[training], all_y.ix[testing]
    
        # Train the model
        model.fit(X_train, y_train)
        # Use the model to predict output
        y_fitted = model.predict(X_test)
        RMSE.append(np.sqrt(mean_squared_error(y_test, y_fitted)))
    # Leave the model fit to the entire dataset
    model.fit(all_X, all_y)
    # And return the array of root mean squared errors
    return RMSE

### first try to generalize the temporalKfold as a class
* base this upon the KFold(_BaseKfold): class in scikitlearn
* returns an iterator

In [ ]:
from sklearn.cross_validation import _BaseKFold

In [ ]:
class windowKFold(_BaseKFold):
    """windowed or temporal K-Folds cross validation iterator.
    Provides train/test indices to split data in train test sets. Split
    dataset into k consecutive folds (without shuffling).
    Each fold is then used a validation set once while the k - 1 remaining
    fold form the training set.
    Parameters
    ----------
    n : int
        Total number of elements.
    n_folds : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle the data before splitting into batches.
    random_state : None, int or RandomState
        Pseudo-random number generator state used for random
        sampling. If None, use default numpy RNG for shuffling
    Examples
    --------
    >>> from sklearn import cross_validation
    >>> X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
    >>> y = np.array([1, 2, 3, 4])
    >>> kf = cross_validation.KFold(4, n_folds=2)
    >>> len(kf)
    2
    >>> print(kf)  # doctest: +NORMALIZE_WHITESPACE
    sklearn.cross_validation.KFold(n=4, n_folds=2, shuffle=False,
                                   random_state=None)
    >>> for train_index, test_index in kf:
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [2 3] TEST: [0 1]
    TRAIN: [0 1] TEST: [2 3]
    Notes
    -----
    The first n % n_folds folds have size n // n_folds + 1, other folds have
    size n // n_folds.
    See also
    --------
    StratifiedKFold: take label information into account to avoid building
    folds with imbalanced class distributions (for binary or multiclass
    classification tasks).
    """

    def __init__(self, n, n_folds=3, indices=None, shuffle=False,
                 random_state=None):
        super(KFold, self).__init__(n, n_folds, indices, shuffle, random_state)
        self.idxs = np.arange(n)
        if shuffle:
            rng = check_random_state(self.random_state)
            rng.shuffle(self.idxs)

    def _iter_test_indices(self):
        n = self.n
        n_folds = self.n_folds
        fold_sizes = (n // n_folds) * np.ones(n_folds, dtype=np.int)
        fold_sizes[:n % n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            yield self.idxs[start:stop]
            current = stop

    def __repr__(self):
        return '%s.%s(n=%i, n_folds=%i, shuffle=%s, random_state=%s)' % (
            self.__class__.__module__,
            self.__class__.__name__,
            self.n,
            self.n_folds,
            self.shuffle,
            self.random_state,
        )

    def __len__(self):
        return self.n_folds


In [ ]:
print("Size of the input set:", input_data.shape)

models = dict(
    logistic = linear_model.LogisticRegression(),
    gbc = ensemble.GradientBoostingClassifier(max_depth=5),
    ridge = linear_model.RidgeClassifier(),
    tree = tree.DecisionTreeClassifier(max_depth=5),
    #svc = svm.LinearSVC(),
    naive_bayes = naive_bayes.MultinomialNB(),  # Can only use if all inputs are positive
    random_forest = ensemble.RandomForestClassifier(n_estimators=10, max_depth=5)
)

win = (spread > 0).astype(int)
rmses = {}
for name, model in models.items():
    rmses[name] = perform_kfold_cross_validation(model, input_data, win, k=3)
    
pd.DataFrame(rmses).boxplot(vert=False, return_type='axes')
plt.gcf().set_size_inches(9, 5)
plt.xlabel("Error in prediction"); plt.ylabel("Model")
plt.show()

## try building a model with deeper splits? or more trees?

In [ ]:
np.shape(X5fold)

In [ ]:
columns_for_modeling

In [ ]:
jl.dump(five_rf_mdl_100A,'frf100.pkl')

In [ ]:
%cd ../
%cd '/home/kesj/work/hrsepara/eda/jl_5yr_100'

In [ ]:
5.4 * 1000/250.

In [ ]:

five_rf_mdl = []
for i in xrange(0,len(tfold5)):
    train_y = y5fold[i].flatten()#[tfold5[i][0]]
    #train_X = #X5fold[tfold5[i][0],:,i]
    train_X = X5fold[:,:,i]
    rfmdl = ensemble.RandomForestClassifier(n_jobs=50,n_estimators=500,max_features=None)
    rfmdl.fit(train_X,train_y)
#baseline_singleRFC = ensemble.RandomForestClassifier(n_jobs=50,n_estimators=500,max_features=None)
#baseline_singleRFC.fit(X,y_term)
#baseline_singleRFC_importances= baseline_singleRFC.feature_importances_
    five_rf_mdl.append(rfmdl)

In [ ]:
(evalmod['hire_tstmp']-evalmod['birth_tstmp'])/np.timedelta64(1,'Y')

In [ ]:
eval_pred_5class, eval_pred_proba5 = evaluate_models(five_rf_mdl,X5eval)

In [ ]:
eval_pred5class_100A, eval_pred5proba = evaluate_models(five_rf_mdl_100A,X5eval)

In [ ]:
len(eval_pred_5class), len(y5eval)

In [ ]:
plot_conf_matrix(y5eval,map(int,eval_pred_5class.mean(axis=1)))

In [ ]:
plot_conf_matrix(y5eval,map(int,eval_pred5class_100A.mean(axis=1)))

In [ ]:
#plot_roc_curve(y3eval,eval_prediction_proba3[:,])
plot_roc_curve(y5eval,eval_pred5proba[:,:,:].mean(axis=2))
plt.ylim([0,1.01])

In [ ]:
plot_conf_matrix(y5eval,map(int,eval_pred_5class.mean(axis=1)+0.3))

In [ ]:
from sklearn.externals import joblib
joblib.dump(five_rf_mdl,'five_rf_mdljl.pkl')

In [ ]:
%cd five_rf_mdljl


In [ ]:
frf= joblib.load('five_rf_mdljl.pkl')

In [ ]:
Beval_pred_5class, Beval_pred_proba5 = evaluate_models(frf,X5eval)

In [ ]:
#plot_roc_curve(y3eval,eval_prediction_proba3[:,])
plot_roc_curve(y5eval,Beval_pred_proba5[:,:,:].mean(axis=2))
plt.ylim([0,1.01])

In [ ]:
Beval_pred5class0, Beval_pred5proba0 = evaluate_models(frf[0],X5eval)

In [ ]:
Beval_pred_proba5[:,:,0]

In [ ]:
#plot_roc_curve(y3eval,eval_prediction_proba3[:,])
plot_roc_curve(y5eval,Beval_pred_proba5[:,:,4])
plt.ylim([0,1.01])

## Try again with joblib directly

In [ ]:
import joblib as jl

In [ ]:
%cd ../
!mkdir jl_5yr


In [ ]:
jldir = '/home/kesj/work/hrsepara/eda/jl_5yr/' 
os.chdir(jldir)

## repeat for 3 year

In [ ]:
tfold3,tfold3_times,panel3term,X3fold,y3fold = setup_tfold_models(em2mod,3,columns_for_modeling)
print len(tfold3)

In [ ]:
%%time
three_rf_mdl_100 = []
for i in xrange(0,len(tfold3)):
    train_y = y3fold[i].flatten()
    train_X = X3fold[:,:,i]
    rfmdl = ensemble.RandomForestClassifier(n_estimators=100,max_features='auto',n_jobs=1)
    rfmdl.fit(train_X,train_y)
#baseline_singleRFC = ensemble.RandomForestClassifier(n_jobs=50,n_estimators=500,max_features=None)
#baseline_singleRFC.fit(X,y_term)
#baseline_singleRFC_importances= baseline_singleRFC.feature_importances_
    three_rf_mdl_100.append(rfmdl)

## dump the results

In [ ]:
%cd '../'
!mkdir 'jl_3yr_100'
%cd '/home/kesj/work/hrsepara/eda/jl_3yr_100'
three_100_list = jl.dump(three_rf_mdl_100,'trf100.pkl')

In [ ]:
len(three_100_list)

In [ ]:
X3eval, y3eval = adjust_eval_by_x_years(evalmod,3,columns_for_modeling)

In [ ]:
eval_pred3class,eval_pred3proba = evaluate_models(three_rf_mdl_100,X3eval)

In [ ]:
plot_roc_curve(y3eval,eval_pred3proba[:,:,:].mean(axis=2))
plt.ylim([0,1.01])

In [ ]:
#plot_conf_matrix(y3eval,eval_)
plot_conf_matrix(y3eval,map(int,eval_pred3class.mean(axis=1)))

In [ ]:
empl

In [ ]:
# omit those that have more than 25% missing:
missing_threshold = 0.25
columns_to_omit = list(sdf[sdf['x_missing'] > missing_threshold].Column.values)
print len(columns_to_omit)
print columns_to_omit

## Construct a simple DataFrame of employee dates using Timestamps

In [ ]:
## Let me return to removing columns I don't want
* keep 

In [ ]:

columns_to_remove = ['']

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

In [ ]:
pipeline = Pipeline([("DF_Converter", br.DataFrameConverter(columns=X.columns)),
                     ("Cat_Converter", br.ConvertCategorical(categorical_columns=categorical_columns)),
                     ("Impute", br.ImputeData()),
                     ("clf", RandomForestClassifier(n_jobs=50))])
pipeline.fit(X, y)
pipeline.predict_proba(X_2014_validation)

## Add/replace some relevant columns to this dataframe
* POSTAL_SFI --> zip5 
* unempl_rate by joining on unemployment
* Age_hire
* terminated 
* sep_status

use a parallel dataframe for dates/timestamps

In [ ]:
class ColumnSelector(TransformerMixin):
    """ Selects column(s) from a pandas DataFrame
    """
    def __init__(self,cols):
        self.cols = cols
    def tranform(self, X, y=None):
        return X[:,self.cols]
    def fit(self, X, y=None):
        return self
    

In [ ]:
## Per discussion with HR COE team decide to truncate data after a particular date


### Helper Functions Follow
* Most taken from ```bear.py```
* 

## Note make sure you have run kinit before the following

In [ ]:
# now assign these values to the arrays
emB['med_surv'] = emB['JOBCODE'].apply(lambda x: med_survival[x] )
emB_eval['med_surv'] = emB_eval['JOBCODE'].apply(lambda x: med_survival[x] )

In [ ]:
modeling_cols

In [ ]:
print len(modeling_cols)
sm_mod_cols = [a for a in modeling_cols]
job_function_cols = [b for b in modeling_cols if b.startswith('JOB_FUNCTION')]
job_function_cols
#for a in ['JOBCODE','grade_code','loc_descr','job_fcode','']
sm_mod_cols.remove('JOBCODE')
sm_mod_cols+=['med_surv']
print len(sm_mod_cols), len(modeling_cols)

In [ ]:
Xp = emB[sm_mod_cols].as_matrix().astype(np.float)
print np.shape(Xp)
#y_tenure = emB.Tenure_years.as_matrix().astype(np.float)
Xeval_p = emB_eval[sm_mod_cols].as_matrix().astype(np.float)


In [ ]:
%%time 
RFRforest2 = ensemble.RandomForestRegressor(n_jobs=50,n_estimators=500,max_features=None)
RFRforest2.fit(Xp,y_tenure)
#importances= forest.feature_importances_
## apply to the eval subset from emB
pred_tenure_eval_2 = RFRforest2.predict(Xeval_p)
np.shape(pred_tenure_eval_2)#, np.shape(y_tenure_class.ix[eval_index])


In [ ]:
graph_feature_importances(RFRforest2,sm_mod_cols)

## working on using the pickled version

In [120]:
# define the application of a series of time-based model
#pred_years = map(np.str,np.arange(1,6))

def apply_survival_models(df,MDL_BASE_PATH,pkl_name,pred_years):
    #proba_df = []
    #import joblib as jl
    import pickle
    import pandas as pd
    mdl_name = MDL_BASE_PATH+pkl_name
    #print mdl_name
    
    
    
    try:
        stored_mdl = pickle.load(open(mdl_name,'rb'))
        #stored_mdl = jl.load(mdl_name)
                
        sf_pred = stored_mdl.predict_survival_function(df)
        sf_pred.index.name='years'
        sf_pred.reset_index(inplace=True)
        pred_proba = get_survival_prediction(df,sf_pred,yr_vals=pred_years)
        
        #df.loc[edf.index[employee_id]]=prob_vals
        #except ValueError: # case when not found (because too young)
        #    df.loc[edf.index[employee_id]]=0.0 # zero probability
            #print employee_id, edf.index[employee_id]
    #df.fillna(0.0,inplace=True) # fillin missing values with zero
    #df[df<0] = 0.0 #replace negative probabilities with zero
        
    except IOError:
        print "requested pickle file not found {0}, using default parameters.".format(mdl_name)
        # TODO create default predictions from simple Survival model for retirement.
        pred_proba = pd.DataFrame()
        
    return pred_proba

In [139]:
tmp_pred = ret_model.predict_survival_function(em2002.tail())
tmp_pred.index.name='years'
tmp_pred.reset_index(inplace=True)
tmp_pred.head()

,years,134261,134262,134263,134264,134265
0,0.000000,1,1,1,1,1
1,50.001369,1,1,1,1,1
2,50.004107,1,1,1,1,1
3,50.006845,1,1,1,1,1
4,50.009582,1,1,1,1,1


In [147]:
from scipy import interpolate
for employee_id,age in enumerate(em2002.Age_years.tail()):
    print employee_id, age
    if age< 50.0:
        proba
    for idx,year in enumerate(yr_range):
        ck_yr = age+year
        
        prior_idx = np.where(tmp_pred.years < ck_yr)[0][-1]
        posterior_idx = np.where(tmp_pred.years > ck_yr)[0][0]
        print idx,year,prior_idx, posterior_idx
        x= [tmp_pred.ix[prior_idx]['years'], tmp_pred.ix[posterior_idx]['years']]
        y= [tmp_pred.ix[prior_idx][employee_id],tmp_pred.ix[posterior_idx][employee_id]]
        #print x,y, year
                ## now interpolate these
        y_interp = interpolate.interp1d(x,y)
        proba = y_interp(ck_yr)
#prior_idx = np.where(psf.years < ck_yr)[0][-1]
#                posterior_idx = np.where(psf.years > ck_yr)[0][0]

0 40.3285420945
0 1 0 1
1 2 0 1
2 3 0 1
3 4 0 1
4 5 0 1
1 51.3976728268
0 1 876 877
1 2 1241 1242
2 3 1606 1607
3 4 1970 1972
4 5 2336 2337
2 38.1437371663
0 1 0 1
1 2 0 1
2 3 0 1
3 4 0 1
4 5 0 1
3 29.5578370979
0 1 0 1
1 2 0 1
2 3 0 1
3 4 0 1
4 5 0 1
4 27.5975359343
0 1 0 1
1 2 0 1
2 3 0 1
3 4 0 1
4 5 0 1


In [180]:
def get_survival_prediction(edf,psf, age_col='Age_years',yr_vals=[1,2,3,4,5]):
    from scipy import interpolate
    
    pred_col_names = ['proba_'+str(y) for y in yr_vals]
    df = pd.DataFrame(index=edf.index,columns=pred_col_names,dtype=float)
    print pred_col_names
    for employee_id,age in enumerate(edf['Age_years']):
        if employee_id % 5000 ==0 :
            print employee_id, age
            
        prob_vals =np.ones((len(yr_vals),))
        #print prob_vals,"\n"
        if age >= 50.0: # only deal with over 50
            for idx,year in enumerate(yr_vals):
                ck_yr = age+year
                try:
                    prior_idx = np.where(psf.years < ck_yr)[0][-1]
                    posterior_idx = np.where(psf.years > ck_yr)[0][0]
                    #if employee_id == 0:
                    #    print prior_idx, posterior_idx, employee_id
                        
                    x= [psf.ix[prior_idx]['years'], psf.ix[posterior_idx]['years']]
                    y= [psf.ix[prior_idx][employee_id+1],psf.ix[posterior_idx][employee_id+1]]
                    # there is an extra column in my processing
                    #print x,y, year
                    ## now interpolate these
                    y_interp = interpolate.interp1d(x,y)
                    proba = y_interp(ck_yr)
                    #print proba
                except IndexError:
                    proba = psf.ix[np.where(psf.years == ck_yr)[0]][employee_id+1]
            
                prob_vals[idx] = proba
                    
        df.loc[edf.index[employee_id]]=prob_vals
    df.fillna(1.0,inplace=True) # fillin missing values with zero
    df[df<0] = 0.0 #replace negative probabilities with zero
    
    return 1-df # convert to probability of retirement

In [171]:
#for a,b in 
o50eval['Age_years'].tail().as_matrix()
#    print a, b

array([ 60.94729637,  61.38809035,  61.06502396,  67.07734428,  51.17316906])

In [ ]:
    """
    for idx,pyr in enumerate(pred_years):
        #pkl_name ='rf'+pyr+'.pkl'#5.pkl'
        #if pcase == 'ret/':
        #    pkl_name = 'R'+pkl_name

        #path_name =MODEL_BASE_PATH+pcase+pyr+'/'
        #print path_name, pkl_name

        # load a model, evaluate it and return the 'average' probability for each person
        #abs_mdl_name = os.path.abspath(path_name+pkl_name)
        #mdl_name = path_name+pkl_name
        #stored_mdl = jl.load(mdl_name)
        
        stored_prediction = evaluate_models(stored_mdl,X)#stored_pred,stored_pred_proba = evaluate_models(stored_mdl,X)

        df = pd.DataFrame(stored_prediction.T)
        df.columns=[pcase[:-1]+pyr+'yr']

        proba_df.append(df)

    pred_df = pd.concat([a for a in proba_df], axis=1)
    """

In [126]:
em2002[['Age_years','retired']].tail()

,Age_years,retired
134261,40.328542,0
134262,51.397673,0
134263,38.143737,0
134264,29.557837,0
134265,27.597536,0


In [161]:
o50eval['Age_years'].tail()

108303    60.947296
23224     61.388090
54614     61.065024
69730     67.077344
101867    51.173169
Name: Age_years, dtype: float64

In [160]:
tmp2 = ret_model.predict_survival_function(o50eval.tail())
tmp2.index.name = 'years'
tmp2.reset_index(inplace=True)
tmp2.head()

,years,108303,23224,54614,69730,101867
0,0.000000,1,1,1,1,1
1,50.001369,1,1,1,1,1
2,50.004107,1,1,1,1,1
3,50.006845,1,1,1,1,1
4,50.009582,1,1,1,1,1


In [162]:
np.where(tmp2.years < 61.947296)[0][-1]

4357

In [184]:
yr_range

array([1, 2, 3, 4, 5])

In [181]:
MDL_BASE_PATH = '/home/kesj/work/hrsepara/proc/'
yr_range = np.arange(1,6)
ret_proba_df2 = apply_survival_models(o50eval.tail(),MDL_BASE_PATH,'retirement_sfA.pkl',yr_range)

['proba_1', 'proba_2', 'proba_3', 'proba_4', 'proba_5']
0 60.9472963723


In [182]:
ret_proba_df2.head()

,proba_1,proba_2,proba_3,proba_4,proba_5
108303,0.435796,0.659508,0.749876,0.807650,0.883177
23224,0.376115,0.478729,0.555260,0.639479,0.724056
54614,0.606475,0.769713,0.846657,0.898669,0.948265
69730,0.765153,0.798855,0.825110,0.856206,0.874256
101867,0.000197,0.000264,0.000458,0.016617,0.049845


In [175]:
o50eval[['Age_years','retired']].tail()

,Age_years,retired
108303,60.947296,0
23224,61.388090,0
54614,61.065024,1
69730,67.077344,0
101867,51.173169,0


In [179]:
current = em2002[em2002.status==0].copy()
len(current)

69475

In [183]:
current_ret_prob = apply_survival_models(current,MDL_BASE_PATH,'retirement_sfA.pkl',yr_range)

['proba_1', 'proba_2', 'proba_3', 'proba_4', 'proba_5']
0 58.4284736482
5000 48.4188911704
10000 48.6488706366
15000 37.6536618754
20000 52.2737850787
25000 50.2395619439
30000 44.9609856263
35000 25.3388090349
40000 56.0246406571
45000 36.2518822724
50000 49.3059548255
55000 31.8466803559
60000 48.0492813142
65000 36.4325804244
